In [2]:
#@title
from __future__ import division
import pickle
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
#from sentence_transformers import SentenceTransformer, util
#import torch
import pandas as pd
import os
import re
import gensim
from collections import Counter
import plotly.express as px
from plotly.offline import plot
from sklearn.cluster import MiniBatchKMeans
import pandas as pd
import itertools
from bs4 import BeautifulSoup as soup
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short,strip_non_alphanum, strip_tags,strip_multiple_whitespaces, preprocess_documents, preprocess_string, strip_numeric, remove_stopwords, strip_tags, strip_punctuation, stem_text
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser
import numpy as np
import matplotlib.pyplot as plt


def flatten_list(somelist):
        if any(isinstance(el, list) for el in somelist) == False:
            return somelist
        flat_list = list(itertools.chain(*somelist))
        return flat_list


def open_pickle(fname):
    with open(fname,'rb') as s:
        data = pickle.load(s)
    return data

def save_pickle(f, filename):
    with open(filename, 'wb') as h:
        pickle.dump(f,h,2)

def remove_urls (text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|-)*\b|http.*?(?=\s)', '', text, flags=re.MULTILINE)
    text = re.sub(r'(www\.)?:\/\/(\w|\.|\/|\?|\=|\&|\%|-)*\b|http.*?(?=\s)', '', text, flags=re.MULTILINE)

    return(text)

def remove_mentions (vTEXT):
    vTEXT = re.sub(r'(@\w+)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_smartquotes(s):
    return re.subn(r'”|“|‘|’','',s)[0]

CUSTOM_FILTERS = [lambda x: x.lower(), remove_urls, remove_mentions, strip_tags,
                  strip_punctuation, remove_stopwords, strip_numeric, strip_short,
                  remove_smartquotes,strip_multiple_whitespaces,stem_text, strip_non_alphanum]

def strip_quoted_post(post):
  if re.search('Quote:',post):
    try:
      s = post.split('Posted by')[1]
      f = s.split('</a>')[1]
    except IndexError:

      f = post.split('</a>')[0]

    return f
  else:
    return post

def normalize_text(text):
    page = soup(text)
    anchors = page.findAll('a')
    for a in anchors:
        a.decompose()

    strongs = page.findAll('strong')
    for s in strongs:
        s.decompose()

    t = remove_urls(page.text)
    ft = re.subn(r'Quote:|Originally Posted by|<a href=|>|</a|\n|\t|\r','',t)[0].strip()
    return ft

In [3]:
import plotly
import networkx as nx
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

def draw_networkx_plotly(GRAPH,title='Network Graph',layout='spring_layout',scale=1,):
  """
  Parameters: GRAPH: NetworkX graph.
              layout: str: "spring_layout" or "spectral_layout" (default="spring_layout")
              scale: int: multiplier to scale node size by degree.

  Returns: plotly plot of network.
  """
  if layout == 'spring_layout':
    pos=nx.spring_layout(GRAPH,weight='weight')
  elif layout == 'spectral_layout':
    pos=nx.spectral_layout(GRAPH,weight='weight')

  edge_x = []
  edge_y = []
  for edge in GRAPH.edges():
      x0, y0 = pos[edge[0]]
      x1, y1 = pos[edge[1]]
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=0.5, color='#888'),
      hoverinfo='none',
      mode='lines')

  node_x = []
  node_y = []
  for node in GRAPH.nodes():
      x, y = pos[node]
      node_x.append(x)
      node_y.append(y)

  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      hovertext=[node for node in list(GRAPH.nodes())],
      marker=dict(
          showscale=False,
          # colorscale options
          #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
          #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
          #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
          colorscale='Picnic',
          reversescale=True,

          color=[node[1]['color'] for node in GRAPH.nodes(data=True)],
          size=[GRAPH.degree[node] * scale for node in GRAPH.nodes],
          colorbar=dict(
              thickness=15,

              xanchor='left',
              titleside='right'
          ),
          line_width=2))

  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  title=title,
                  titlefont_size=16,
                  showlegend=False,
                  hovermode='closest',

                  margin=dict(b=20,l=5,r=5,t=40),
                  annotations=[ dict(
                      text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                      showarrow=False,
                      xref="paper", yref="paper",
                      x=0.005, y=-0.002 ) ],
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  plot(fig)


In [4]:
import re, urllib, bleach
from pandas import DataFrame
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup as bsoup
import ast
import time
import itertools
import json, csv, sqlite3
import pandas as pd
from itertools import chain
import numpy as np
from scipy import stats
from collections import defaultdict
#import selenium
from natsort import natsorted
#from selenium import webdriver
from natsort import natsorted
import copy

from sentence_transformers import SentenceTransformer

smodel = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

def flatten_list(somelist):
        if any(isinstance(el, list) for el in somelist) == False:
            return somelist
        flat_list = list(itertools.chain(*somelist))
        return flat_list

def citydata_scrape_local(urls):
    pages = []
    for url in urls:
        try:
            page = urllib.request.urlopen(url)
            pages.append(page)
        except (IOError, AttributeError):
            pass
    soups = [bsoup(pages[i]) for i in range(len(pages))]
    tables = city_data_tables(soups)

    #get timestamps
    timestamps = [table.findAll('div',{'class':'normal',}) for table in tables]
    datetime = []
    for ts in timestamps:
        datetime.append(re.findall(r'[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9],+\s+[0-9][0-9]:[0-9][0-9]+\s+['
                                        r'A-Z]['
                                  r'A-Z]', str(ts)))
    dates = []
    for date in datetime:
        if date == []:
            dates.append('')
        else:
            dates.append(date[0])
    #get posts
    posts_init = [s.findAll("td", {"id": lambda L: L and L.startswith('td_post_')}) for s in tables]
    #get post ids
    pattern = re.compile('div id="[^"]*"')
    div_ids = [s.findAll("td", {"id": lambda L: L and L.startswith('td_post_')}) for s in tables]
    ids_init = []
    for div in div_ids:
        ids_init.append(re.findall('\d+',str(pattern.findall(str(div)))))

    #get quoted post
    quoted_init = [post.findAll('td',{'class':'alt2','style':"border:1px inset"}) for post in tables]  #quoted json
    quoted = []
    for qi in quoted_init:
        if re.search('Originally Posted', str(qi)):
            quoted.append(qi[0])
        else:
            quoted.append('')
    #get quote id
    quote_ids = [get_quote_id(quote) for quote in quoted]
    qb = []
    for quote in quote_ids:
        if quote == []:
            qb.append('')
        else:
            qb.append(str(flatten_list(quote)[0]))
    
    df = pd.DataFrame()
    df['post_id'] = [str(i[0])[1:-1] for i in ids_init]
    df['post_body'] = [str(p[0])[1:-1] for p in posts_init]
    df['post'] = [normalize_text(str(p)) for p in posts_init]
    df['datetime'] = pd.to_datetime(dates)
    df['quote_id'] = qb
    df['quote_body'] = quoted
    df['quote'] = [normalize_text(str(quote)) for quote in quoted]
    return df

# def cityscrape_from_url_dataframe(urls):

#     pages = []
#     for url in urls:
#          try:
#              page = urllib.request.urlopen(url)

#              pages.append(page)
#          except (IOError, AttributeError):
#              pass


#     soups = [bsoup(pages[i]) for i in range(len(pages))]

#     return soups
#     tables = flatten_list([s.findAll('table') for s in soups])
    
#     #city_data post meta
#     ids = city_data_ids(soups)
#     dates = city_data_datetime(soups) #datetime json

    
#     # names = city_data_names(soups) #member json

#     #city_data post content
#     posts =  city_data_posts(soups)
    
    

#     #cleaned_posts = [re.subn('\n|\t', ' ', bleach.clean(posts[i].text, tags=[], attributes=[], styles=[], strip=True))[0].encode('ascii', 'ignore') for i in range(len(posts))]

#     quoted_init = [post.findAll('td',{'class':'alt2','style':"border:1px inset"}) for post in posts]  #quoted json
#     quoted = []
#     for qi in quoted_init:
#         if re.search('Originally Posted', str(qi)):
#             quoted.append(qi)
#         else:
#             quoted.append([])

#     quote_ids = [get_quote_id(quote) for quote in quoted]

#     #create dataframe for export
#     data = pd.DataFrame()
#     data['post_id'] = ids
#     #data['quote_id'] = quote_ids
#     #data['datetime'] = dates
#     #data['raw'] = raws
#     #data['quote_body'] = quoted
#     #data['cleaned_quotes'] = cleaned_quotes
#     #data['post'] = [normalize_text(str(p)) for p in posts]
#     data['post_body'] = posts

# #    data['links'] = links
# #    data['video'] = videos
# #    data['images'] = images



#     #data.to_csv('city_data0.csv')
#     return data

def add_cleaned_posts(dataframe):
    cposts = dataframe['post_body'].tolist()
    cleaned_posts = [bleach.clean(str(remove_table(bsoup(post))), strip=True) for post in cposts]
    cp = [re.subn('Quote:','',p)[0] for p in cleaned_posts]
    dataframe['cleaned_post'] = cp
    return dataframe

def get_quote_id(quote):
    qi = []
    q = str(quote).split()
    for i in q:
        if re.search('href=', i):
            qi.append(i.split('#post')[-1])
        else:
            pass

    qid  =[]
    if len(qi) > 0:
        qid.append(re.subn('"','',qi[0])[0])
    else:
        qid.append('')
    return qid



def page_save(urls,t=3):
    content = []
    browser = webdriver.Firefox()
    for url in urls:
        try:
            browser.get(url)
            time.sleep(t)
            content.append(browser.page_source)

            print(url + ' downloaded!')

        except:
            print('WebDriverException')

    return content

def remove_table(tagset):
    for t in tagset.findAll('table'):
        t.decompose()
    return tagset

def datetime_post_count_data(dataframe):
    """takes dataframe from city_data scrape as args"""

    #create new dataframe
    df = DataFrame()

    #add post count data; each date = 1 post
    df['post_count'] = [1 for i in range(len(dataframe))]

    # reformat city_data datetime to pandas datetime
    df.index = pd.to_datetime(dataframe['datetime'])

    return df

def city_data_video_src(listofvideos):
    """helper function that takes lists of iframe links per city_data post; requires that a list of videos per post
    be created first"""
    sources = []
    if len(listofvideos) > 0:
        for v in listofvideos:

            sources.append(v['src'])

    else:
        sources.append([])
    return sources

def city_data_tables(soups):
    # if len(soups) < 3:
    #     tables_init = soup.findAll('table', {"id": lambda L: L and L.startswith('post')})
    # else:
    tables_init = [s.findAll("table", {"id": lambda L: L and L.startswith('post')}) for s in soups]
    tables = []
    for table in tables_init:
        if table not in tables:
            tables.append(table)

    return flatten_list(tables)

def city_data_names(soups):
    # if len(soups) < 3:
    #     tables_init = soup.findAll('table', {"id": lambda L: L and L.startswith('post')})
    # else:

    names_init = [s.findAll("a", {"class": "bigusername"}) for s in soups]
    names = [bleach.clean(n, tags=[], attributes=[], styles=[], strip=True) for n in flatten_list(names_init)]


    return names

def city_data_ids(soups):
    pattern = re.compile('div id="[^"]*"')
    # if len(soups) < 3:
    #     div_ids = soups.findAll("div", {"id": lambda L: L and L.startswith('post_message_')})
    # else:
    div_ids = [s.findAll("td", {"id": lambda L: L and L.startswith('td_post_')}) for s in soups]
    ids_init = []
    for div in div_ids:
        ids_init.append(re.findall('\d+',str(pattern.findall(str(div)))))

    ids = []
    for id in ids_init:
        if id not in ids:
            ids.append(id)

    return flatten_list(ids)


def city_data_posts(soups):
    # if len(soups) < 3:
    #     posts_init = soups.findAll('div', {"id": lambda L: L and L.startswith('post_message_')})
    # else:
    #posts_init = [s.findAll("div", {"id": lambda L: L and L.startswith('post_message_')}) for s in soups] # collect
    posts_init = [s.findAll("td", {"id": lambda L: L and L.startswith('td_post_')}) for s in soups] # collect

    #beware of duplicates!
    posts = []  # post_content json
    for p in posts_init:
        if p not in posts:
            posts.append(p)

    return flatten_list(posts)

def city_data_datetime(soups):
    
    # if len(soups) < 3:
    #     timestamps = soups.findAll('td',{'class':'thead'})
    # else:
    #tds = [s.findAll('td',{'class':'thead'}) for s in soups]
    #posts = city_data_posts(soups)
    timestamps = [s.findAll('div',{'class':'normal','style': 'color:#A2B3D0;'}) for s in soups]
    datetime_init = []

    for ts in timestamps:
        datetime_init.append(re.findall(r'[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9],+\s+[0-9][0-9]:[0-9][0-9]+\s+['
                                        r'A-Z]['
                                  r'A-Z]', str(ts)))
    datetime = []
    for date in datetime_init:
        if date not in datetime:
            datetime.append(date)
            

    return datetime






C:\Users\rmomi\anaconda3\envs\bertopic\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [5]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser

def get_topical_coherence(texts,topics, metric='u_mass'):
    if type(texts[0]) == str:
        #train dictionary on tokenized documents (corpus)
        doc_tokens = [text_tokenize(text) for text in texts]
        phrases = Phrases(doc_tokens, min_count=1, threshold=1)
        bigrams = phrases[doc_tokens]
        dct = Dictionary(bigrams) 
        
        
        #topics = list of topical words
        #texts = tokenized documents
        #corpus = doc2bow
        #dictionary = gensim dictionary
        cm = CoherenceModel(topics=topics,texts=texts, corpus=[dct.doc2bow(c) for c in doc_tokens], dictionary=dct, coherence=metric)
        return cm.get_coherence()
    elif type(texts[0]) == list:
        doc_tokens = texts
        phrases = Phrases(doc_tokens, min_count=1, threshold=1)
        bigrams = phrases[doc_tokens]
        dct = Dictionary(bigrams) 
        
        
        #topics = list of topical words
        #texts = tokenized documents
        #corpus = doc2bow
        #dictionary = gensim dictionary
        cm = CoherenceModel(topics=topics,texts=texts, corpus=[dct.doc2bow(c) for c in doc_tokens], dictionary=dct, coherence=metric)
        return cm.get_coherence()
        

def get_topical_terms(dataframe, term_count=25):
    topics = []
    for i in range(len(dataframe.columns)-1):
        
        terms = dataframe.sort_values(by='coef_'+str(i),ascending=False)[:term_count]['term'].tolist()
        topics.append(terms)
    filtered = []
    for topic in topics:
        t = []
        for term in topic:
            if len(term.split()) > 1:
                t.append('_'.join(term.split()))
            else:
                t.append(term)
        filtered.append(t)
    return filtered
import numpy as np

from scipy.spatial import distance
from itertools import combinations


def proportion_unique_words(topics, topk=10):
    """
    compute the proportion of unique words

    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity will be computed
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than '+str(topk))
    else:
        unique_words = set()
        for topic in topics:
            unique_words = unique_words.union(set(topic[:topk]))
        puw = len(unique_words) / (topk * len(topics))
        return puw


def irbo(topics, weight=0.9, topk=10):
    """
    compute the inverted rank-biased overlap

    Parameters
    ----------
    topics: a list of lists of words
    weight: p (float), default 1.0: Weight of each
        agreement at depth d:p**(d-1). When set
        to 1.0, there is no weight, the rbo returns
        to average overlap.
    topk: top k words on which the topic diversity
          will be computed

    Returns
    -------
    irbo : score of the rank biased overlap over the topics
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        collect = []
        for list1, list2 in combinations(topics, 2):
            word2index = get_word2index(list1, list2)
            indexed_list1 = [word2index[word] for word in list1]
            indexed_list2 = [word2index[word] for word in list2]
            rbo_val = rbo(indexed_list1[:topk], indexed_list2[:topk], p=weight)[2]
            collect.append(rbo_val)
        return 1 - np.mean(collect)


def word_embedding_irbo(topics, word_embedding_model, weight=0.9, topk=10):
    '''
    compute the word embedding-based inverted rank-biased overlap

    Parameters
    ----------
    topics: a list of lists of words
    weight: p (float), default 1.0: Weight of each agreement at depth d:
    p**(d-1). When set to 1.0, there is no weight, the rbo returns to average overlap.
    
    Returns
    -------
    weirbo: word embedding-based inverted rank_biased_overlap over the topics
    '''
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        collect = []
        for list1, list2 in combinations(topics, 2):
            word2index = get_word2index(list1, list2)
            index2word = {v: k for k, v in word2index.items()}
            indexed_list1 = [word2index[word] for word in list1]
            indexed_list2 = [word2index[word] for word in list2]
            rbo_val = word_embeddings_rbo(indexed_list1[:topk], indexed_list2[:topk], p=weight,
                                          index2word=index2word, word2vec=word_embedding_model)[2]
            collect.append(rbo_val)
        return 1 - np.mean(collect)


def pairwise_jaccard_diversity(topics, topk=10):
    '''
    compute the average pairwise jaccard distance between the topics 
  
    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity
          will be computed
    
    Returns
    -------
    pjd: average pairwise jaccard distance
    '''
    dist = 0
    count = 0
    for list1, list2 in combinations(topics, 2):
        js = 1 - len(set(list1).intersection(set(list2)))/len(set(list1).union(set(list2)))
        dist = dist + js
        count = count + 1
    return dist/count


def pairwise_word_embedding_distance(topics, word_embedding_model, topk=10):
    """
    :param topk: how many most likely words to consider in the evaluation
    :return: topic coherence computed on the word embeddings similarities
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        count = 0
        sum_dist = 0
        for list1, list2 in combinations(topics, 2):
            count = count+1
            word_counts = 0
            dist = 0
            for word1 in list1[:topk]:
                for word2 in list2[:topk]:
                    dist = dist + distance.cosine(word_embedding_model.wv[word1], word_embedding_model.wv[word2])
                    word_counts = word_counts + 1

            dist = dist/word_counts
            sum_dist = sum_dist + dist
        return sum_dist/count


def centroid_distance(topics, word_embedding_model, topk=10):
    """
    :param topk: how many most likely words to consider in the evaluation
    :return: topic coherence computed on the word embeddings similarities
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        count = 0
        for list1, list2 in combinations(topics, 2):
            count = count + 1
            centroid1 = np.zeros(word_embedding_model.vector_size)
            centroid2 = np.zeros(word_embedding_model.vector_size)
            for word1 in list1[:topk]:
                centroid1 = centroid1 + word_embedding_model[word1]
            for word2 in list2[:topk]:
                centroid2 = centroid2 + word_embedding_model[word2]
            centroid1 = centroid1 / len(list1[:topk])
            centroid2 = centroid2 / len(list2[:topk])
        return distance.cosine(centroid1, centroid2)


def get_word2index(list1, list2):
    words = set(list1)
    words = words.union(set(list2))
    word2index = {w: i for i, w in enumerate(words)}
    return word2index
"""Rank-biased overlap, a ragged sorted list similarity measure.
See http://doi.acm.org/10.1145/1852102.1852106 for details. All functions
directly corresponding to concepts from the paper are named so that they can be
clearly cross-identified.
The definition of overlap has been modified to account for ties. Without this,
results for lists with tied items were being inflated. The modification itself
is not mentioned in the paper but seems to be reasonable, see function
``overlap()``. Places in the code which diverge from the spec in the paper
because of this are highlighted with comments.
The two main functions for performing an RBO analysis are ``rbo()`` and
``rbo_dict()``; see their respective docstrings for how to use them.
The following doctest just checks that equivalent specifications of a
problem yield the same result using both functions:
    >>> lst1 = [{"c", "a"}, "b", "d"]
    >>> lst2 = ["a", {"c", "b"}, "d"]
    >>> ans_rbo = _round(rbo(lst1, lst2, p=.9))
    >>> dct1 = dict(a=1, b=2, c=1, d=3)
    >>> dct2 = dict(a=1, b=2, c=2, d=3)
    >>> ans_rbo_dict = _round(rbo_dict(dct1, dct2, p=.9, sort_ascending=True))
    >>> ans_rbo == ans_rbo_dict
    True
"""

import math
from bisect import bisect_left
from collections import namedtuple


RBO = namedtuple("RBO", "min res ext")
RBO.__doc__ += ": Result of full RBO analysis"
RBO.min.__doc__ = "Lower bound estimate"
RBO.res.__doc__ = "Residual corresponding to min; min + res is an upper bound estimate"
RBO.ext.__doc__ = "Extrapolated point estimate"


def _round(obj):
    if isinstance(obj, RBO):
        return RBO(_round(obj.min), _round(obj.res), _round(obj.ext))
    else:
        return round(obj, 3)


def set_at_depth(lst, depth):
    ans = set()
    for v in lst[:depth]:
        if isinstance(v, set):
            ans.update(v)
        else:
            ans.add(v)
    return ans


def raw_overlap(list1, list2, depth):
    """Overlap as defined in the article.
    """
    set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    return len(set1.intersection(set2)), len(set1), len(set2)


def overlap(list1, list2, depth):
    """Overlap which accounts for possible ties.
    This isn't mentioned in the paper but should be used in the ``rbo*()``
    functions below, otherwise overlap at a given depth might be > depth which
    inflates the result.
    There are no guidelines in the paper as to what's a good way to calculate
    this, but a good guess is agreement scaled by the minimum between the
    requested depth and the lengths of the considered lists (overlap shouldn't
    be larger than the number of ranks in the shorter list, otherwise results
    are conspicuously wrong when the lists are of unequal lengths -- rbo_ext is
    not between rbo_min and rbo_min + rbo_res.
    >>> overlap("abcd", "abcd", 3)
    3.0
    >>> overlap("abcd", "abcd", 5)
    4.0
    >>> overlap(["a", {"b", "c"}, "d"], ["a", {"b", "c"}, "d"], 2)
    2.0
    >>> overlap(["a", {"b", "c"}, "d"], ["a", {"b", "c"}, "d"], 3)
    3.0
    """
    ov = agreement(list1, list2, depth) * min(depth, len(list1), len(list2))
    return ov
    # NOTE: comment the preceding and uncomment the following line if you want
    # to stick to the algorithm as defined by the paper
    # return raw_overlap(list1, list2, depth)[0]


def agreement(list1, list2, depth):
    """Proportion of shared values between two sorted lists at given depth.
    >>> _round(agreement("abcde", "abdcf", 1))
    1.0
    >>> _round(agreement("abcde", "abdcf", 3))
    0.667
    >>> _round(agreement("abcde", "abdcf", 4))
    1.0
    >>> _round(agreement("abcde", "abdcf", 5))
    0.8
    >>> _round(agreement([{1, 2}, 3], [1, {2, 3}], 1))
    0.667
    >>> _round(agreement([{1, 2}, 3], [1, {2, 3}], 2))
    1.0
    """
    len_intersection, len_set1, len_set2 = raw_overlap(list1, list2, depth)
    return 2 * len_intersection / (len_set1 + len_set2)


def cumulative_agreement(list1, list2, depth):
    return (agreement(list1, list2, d) for d in range(1, depth + 1))


def average_overlap(list1, list2, depth=None):
    """Calculate average overlap between ``list1`` and ``list2``.
    >>> _round(average_overlap("abcdefg", "zcavwxy", 1))
    0.0
    >>> _round(average_overlap("abcdefg", "zcavwxy", 2))
    0.0
    >>> _round(average_overlap("abcdefg", "zcavwxy", 3))
    0.222
    >>> _round(average_overlap("abcdefg", "zcavwxy", 4))
    0.292
    >>> _round(average_overlap("abcdefg", "zcavwxy", 5))
    0.313
    >>> _round(average_overlap("abcdefg", "zcavwxy", 6))
    0.317
    >>> _round(average_overlap("abcdefg", "zcavwxy", 7))
    0.312
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    return sum(cumulative_agreement(list1, list2, depth)) / depth


def rbo_at_k(list1, list2, p, depth=None):
    # ``p**d`` here instead of ``p**(d - 1)`` because enumerate starts at
    # 0
    depth = min(len(list1), len(list2)) if depth is None else depth
    d_a = enumerate(cumulative_agreement(list1, list2, depth))
    return (1 - p) * sum(p ** d * a for (d, a) in d_a)


def rbo_min(list1, list2, p, depth=None):
    """Tight lower bound on RBO.
    See equation (11) in paper.
    >>> _round(rbo_min("abcdefg", "abcdefg", .9))
    0.767
    >>> _round(rbo_min("abcdefgh", "abcdefg", .9))
    0.767
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    x_k = overlap(list1, list2, depth)
    log_term = x_k * math.log(1 - p)
    sum_term = sum(
        p ** d / d * (overlap(list1, list2, d) - x_k) for d in range(1, depth + 1)
    )
    return (1 - p) / p * (sum_term - log_term)


def rbo_res(list1, list2, p):
    """Upper bound on residual overlap beyond evaluated depth.
    See equation (30) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible. In particular, for identical lists, ``rbo_min()`` and
    ``rbo_res()`` should add up to 1, which is the case.
    >>> _round(rbo_res("abcdefg", "abcdefg", .9))
    0.233
    >>> _round(rbo_res("abcdefg", "abcdefghijklmnopqrstuvwxyz", .9))
    0.239
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l)
    # since overlap(...) can be fractional in the general case of ties and f
    # must be an integer --> math.ceil()
    f = int(math.ceil(l + s - x_l))
    # upper bound of range() is non-inclusive, therefore + 1 is needed
    term1 = s * sum(p ** d / d for d in range(s + 1, f + 1))
    term2 = l * sum(p ** d / d for d in range(l + 1, f + 1))
    term3 = x_l * (math.log(1 / (1 - p)) - sum(p ** d / d for d in range(1, f + 1)))
    return p ** s + p ** l - p ** f - (1 - p) / p * (term1 + term2 + term3)


def rbo_ext(list1, list2, p):
    """RBO point estimate based on extrapolating observed overlap.
    See equation (32) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible.
    >>> _round(rbo_ext("abcdefg", "abcdefg", .9))
    1.0
    >>> _round(rbo_ext("abcdefg", "bacdefg", .9))
    0.9
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l)
    x_s = overlap(list1, list2, s)
    # the paper says overlap(..., d) / d, but it should be replaced by
    # agreement(..., d) defined as per equation (28) so that ties are handled
    # properly (otherwise values > 1 will be returned)
    # sum1 = sum(p**d * overlap(list1, list2, d)[0] / d for d in range(1, l + 1))
    sum1 = sum(p ** d * agreement(list1, list2, d) for d in range(1, l + 1))
    sum2 = sum(p ** d * x_s * (d - s) / s / d for d in range(s + 1, l + 1))
    term1 = (1 - p) / p * (sum1 + sum2)
    term2 = p ** l * ((x_l - x_s) / l + x_s / s)
    return term1 + term2


def rbo(list1, list2, p):
    """Complete RBO analysis (lower bound, residual, point estimate).
    ``list`` arguments should be already correctly sorted iterables and each
    item should either be an atomic value or a set of values tied for that
    rank. ``p`` is the probability of looking for overlap at rank k + 1 after
    having examined rank k.
    >>> lst1 = [{"c", "a"}, "b", "d"]
    >>> lst2 = ["a", {"c", "b"}, "d"]
    >>> _round(rbo(lst1, lst2, p=.9))
    RBO(min=0.489, res=0.477, ext=0.967)
    """
    if not 0 <= p <= 1:
        raise ValueError("The ``p`` parameter must be between 0 and 1.")
    args = (list1, list2, p)
    return RBO(rbo_min(*args), rbo_res(*args), rbo_ext(*args))


def sort_dict(dct, *, ascending=False):
    """Sort keys in ``dct`` according to their corresponding values.
    Sorts in descending order by default, because the values are
    typically scores, i.e. the higher the better. Specify
    ``ascending=True`` if the values are ranks, or some sort of score
    where lower values are better.
    Ties are handled by creating sets of tied keys at the given position
    in the sorted list.
    >>> dct = dict(a=1, b=2, c=1, d=3)
    >>> list(sort_dict(dct)) == ['d', 'b', {'a', 'c'}]
    True
    >>> list(sort_dict(dct, ascending=True)) == [{'a', 'c'}, 'b', 'd']
    True
    """
    scores = []
    items = []
    # items should be unique, scores don't have to
    for item, score in dct.items():
        if not ascending:
            score *= -1
        i = bisect_left(scores, score)
        if i == len(scores):
            scores.append(score)
            items.append(item)
        elif scores[i] == score:
            existing_item = items[i]
            if isinstance(existing_item, set):
                existing_item.add(item)
            else:
                items[i] = {existing_item, item}
        else:
            scores.insert(i, score)
            items.insert(i, item)
    return items


def rbo_dict(dict1, dict2, p, *, sort_ascending=False):
    """Wrapper around ``rbo()`` for dict input.
    Each dict maps items to be sorted to the score according to which
    they should be sorted. The RBO analysis is then performed on the
    resulting sorted lists.
    The sort is descending by default, because scores are typically the
    higher the better, but this can be overridden by specifying
    ``sort_ascending=True``.
    >>> dct1 = dict(a=1, b=2, c=1, d=3)
    >>> dct2 = dict(a=1, b=2, c=2, d=3)
    >>> _round(rbo_dict(dct1, dct2, p=.9, sort_ascending=True))
    RBO(min=0.489, res=0.477, ext=0.967)
    """
    list1, list2 = (
        sort_dict(dict1, ascending=sort_ascending),
        sort_dict(dict2, ascending=sort_ascending),
    )
    return rbo(list1, list2, p)

import math
from bisect import bisect_left
from collections import namedtuple
from collections import OrderedDict

RBO = namedtuple("RBO", "min res ext")
RBO.__doc__ += ": Result of full RBO analysis"
RBO.min.__doc__ = "Lower bound estimate"
RBO.res.__doc__ = "Residual corresponding to min; min + res is an upper bound estimate"
RBO.ext.__doc__ = "Extrapolated point estimate"

def _round(obj):
    if isinstance(obj, RBO):
        return RBO(_round(obj.min), _round(obj.res), _round(obj.ext))
    else:
        return round(obj, 3)


def set_at_depth(lst, depth):
    ans = set()
    for v in lst[:depth]:
        if isinstance(v, set):
            ans.update(v)
        else:
            ans.add(v)
    return ans


def embeddings_overlap(list1, list2, depth, index2word, word2vec):
    #set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    #return len(set1.intersection(set2)), len(set1), len(set2)

    set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    word_list1 = [index2word[index] for index in list1]
    word_list2 = [index2word[index] for index in list2]

    similarities = {}
    for w1 in word_list1[:depth]:
        for w2 in word_list2[:depth]:
            similarities[(w1,w2)] = word2vec.similarity(w1, w2)

    similarities = OrderedDict(sorted(similarities.items(), key=lambda x: -x[1]))

    e_ov = 0
    key_list = list(similarities.keys())
    for k in key_list:
        if k in similarities.keys():
            #print(k, similarities[k])
            e_ov = e_ov + similarities[k]
            similarities = {save_k: v for save_k, v in similarities.items()
                            if save_k[0] != k[0] and save_k[1] != k[1]}
    #e_ov = 1
    #print("****")
    return e_ov, len(set1), len(set2)


def overlap(list1, list2, depth, index2word, word2vec):
    #return agreement(list1, list2, depth) * min(depth, len(list1), len(list2))
    # NOTE: comment the preceding and uncomment the following line if you want
    # to stick to the algorithm as defined by the paper
    ov = embeddings_overlap(list1, list2, depth, index2word, word2vec)[0]
    return ov


def agreement(list1, list2, depth, index2word, word2vec):
    """Proportion of shared values between two sorted lists at given depth."""
    len_intersection, len_set1, len_set2 = embeddings_overlap(list1, list2, depth, index2word, word2vec)
    return 2 * len_intersection / (len_set1 + len_set2)


def cumulative_agreement(list1, list2, depth, index2word, word2vec):
    return (agreement(list1, list2, d, index2word, word2vec) for d in range(1, depth + 1))


def average_overlap(list1, list2, index2word, word2vec, depth=None):
    """Calculate average overlap between ``list1`` and ``list2``.
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    return sum(cumulative_agreement(list1, list2, depth, index2word=index2word, word2vec=word2vec)) / depth


def rbo_at_k(list1, list2, p, index2word, word2vec, depth=None):
    # ``p**d`` here instead of ``p**(d - 1)`` because enumerate starts at
    # 0
    depth = min(len(list1), len(list2)) if depth is None else depth
    d_a = enumerate(cumulative_agreement(list1, list2, depth, index2word=index2word, word2vec=word2vec))
    return (1 - p) * sum(p ** d * a for (d, a) in d_a)


def rbo_min(list1, list2, p, index2word, word2vec, depth=None):
    """Tight lower bound on RBO.
    See equation (11) in paper.
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    x_k = overlap(list1, list2, depth, index2word, word2vec)
    log_term = x_k * math.log(1 - p)
    sum_term = sum(
        p ** d / d * (overlap(list1, list2, d, index2word, word2vec=word2vec) - x_k) for d in range(1, depth + 1)
    )
    return (1 - p) / p * (sum_term - log_term)


def rbo_res(list1, list2, p, index2word, word2vec):
    """Upper bound on residual overlap beyond evaluated depth.
    See equation (30) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible. In particular, for identical lists, ``rbo_min()`` and
    ``rbo_res()`` should add up to 1, which is the case.
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l, index2word, word2vec)
    # since overlap(...) can be fractional in the general case of ties and f
    # must be an integer --> math.ceil()
    f = int(math.ceil(l + s - x_l))
    # upper bound of range() is non-inclusive, therefore + 1 is needed
    term1 = s * sum(p ** d / d for d in range(s + 1, f + 1))
    term2 = l * sum(p ** d / d for d in range(l + 1, f + 1))
    term3 = x_l * (math.log(1 / (1 - p)) - sum(p ** d / d for d in range(1, f + 1)))
    return p ** s + p ** l - p ** f - (1 - p) / p * (term1 + term2 + term3)


def rbo_ext(list1, list2, p, index2word, word2vec):
    """RBO point estimate based on extrapolating observed overlap.
    See equation (32) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible.
    >>> _round(rbo_ext("abcdefg", "abcdefg", .9))
    1.0
    >>> _round(rbo_ext("abcdefg", "bacdefg", .9))
    0.9
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l, index2word, word2vec)
    x_s = overlap(list1, list2, s, index2word, word2vec)
    # the paper says overlap(..., d) / d, but it should be replaced by
    # agreement(..., d) defined as per equation (28) so that ties are handled
    # properly (otherwise values > 1 will be returned)
    # sum1 = sum(p**d * overlap(list1, list2, d)[0] / d for d in range(1, l + 1))
    sum1 = sum(p ** d * agreement(list1, list2, d, index2word=index2word, word2vec=word2vec)
               for d in range(1, l + 1))
    sum2 = sum(p ** d * x_s * (d - s) / s / d for d in range(s + 1, l + 1))
    term1 = (1 - p) / p * (sum1 + sum2)
    term2 = p ** l * ((x_l - x_s) / l + x_s / s)
    return term1 + term2


def word_embeddings_rbo(list1, list2, p, index2word, word2vec):
    """Complete RBO analysis (lower bound, residual, point estimate).
    ``list`` arguments should be already correctly sorted iterables and each
    item should either be an atomic value or a set of values tied for that
    rank. ``p`` is the probability of looking for overlap at rank k + 1 after
    having examined rank k.
    >>> lst1 = [{"c", "a"}, "b", "d"]
    >>> lst2 = ["a", {"c", "b"}, "d"]
    >>> _round(rbo(lst1, lst2, p=.9))
    RBO(min=0.489, res=0.477, ext=0.967)
    """
    if not 0 <= p <= 1:
        raise ValueError("The ``p`` parameter must be between 0 and 1.")
    args = (list1, list2, p, index2word, word2vec)

    return RBO(rbo_min(*args), rbo_res(*args), rbo_ext(*args))


def sort_dict(dct, *, ascending=False):
    """Sort keys in ``dct`` according to their corresponding values.
    Sorts in descending order by default, because the values are
    typically scores, i.e. the higher the better. Specify
    ``ascending=True`` if the values are ranks, or some sort of score
    where lower values are better.
    Ties are handled by creating sets of tied keys at the given position
    in the sorted list.
    >>> dct = dict(a=1, b=2, c=1, d=3)
    >>> list(sort_dict(dct)) == ['d', 'b', {'a', 'c'}]
    True
    >>> list(sort_dict(dct, ascending=True)) == [{'a', 'c'}, 'b', 'd']
    True
    """
    scores = []
    items = []
    # items should be unique, scores don't have to
    for item, score in dct.items():
        if not ascending:
            score *= -1
        i = bisect_left(scores, score)
        if i == len(scores):
            scores.append(score)
            items.append(item)
        elif scores[i] == score:
            existing_item = items[i]
            if isinstance(existing_item, set):
                existing_item.add(item)
            else:
                items[i] = {existing_item, item}
        else:
            scores.insert(i, score)
            items.insert(i, item)
    return items


def rbo_dict(dict1, dict2, p, index2word, word2vec, *, sort_ascending=False):
    """Wrapper around ``rbo()`` for dict input.
    Each dict maps items to be sorted to the score according to which
    they should be sorted. The RBO analysis is then performed on the
    resulting sorted lists.
    The sort is descending by default, because scores are typically the
    higher the better, but this can be overridden by specifying
    ``sort_ascending=True``.
    """
    list1, list2 = (
        sort_dict(dict1, ascending=sort_ascending),
        sort_dict(dict2, ascending=sort_ascending),
    )
    return word_embeddings_rbo(list1, list2, p, index2word, word2vec)

def get_diversity_scores(topics, model, topn=10, topic_type=''):
    df = pd.DataFrame({"puw:":proportion_unique_words(topics, topk=topn),
"jd:": pairwise_jaccard_diversity(topics, topk=topn),
"we-pd:": pairwise_word_embedding_distance(topics, model, topk=topn),
"we-cd:": centroid_distance(topics, model.wv, topk=topn),
"we-irbo p=0.5:":word_embedding_irbo(topics,model.wv, weight=0.5, topk=topn),
"we-irbo p=0.9:":word_embedding_irbo(topics,model.wv, weight=0.9, topk=topn)},index=[topic_type])
    return df

def get_coherence_diversity_scores(texts, topics, model, topn=10, topic_type='',metric='u_mass'):
    
    df = pd.DataFrame({'u_mass_coherence':get_topical_coherence(texts,topics,metric=metric),
        "puw:":proportion_unique_words(topics, topk=topn),
"jd:": pairwise_jaccard_diversity(topics, topk=topn),
"we-pd:": pairwise_word_embedding_distance(topics, model, topk=topn),
"we-cd:": centroid_distance(topics, model.wv, topk=topn),
"we-irbo p=0.5:":word_embedding_irbo(topics,model.wv, weight=0.5, topk=topn),
"we-irbo p=0.9:":word_embedding_irbo(topics,model.wv, weight=0.9, topk=topn)},index=[topic_type])
    return df


In [6]:
import networkx as nx
def make_graph_citydata(dataframe):
    c_color = {}
    for i in range(24):
        c_color[i] = px.colors.qualitative.Light24[i]
    edges = [(str(x),str(y)) for (x,y) in list(zip(dataframe.post_id.tolist(),dataframe.quote_id.tolist()))]
    G = nx.MultiDiGraph()
    for i in range(len(dataframe)):
        G.add_node(dataframe.iloc[i]['post_id'],text=dataframe.iloc[i]['post'])
                 

    for i in range(len(dataframe)):
        if dataframe.iloc[i]['quote_id'] != '' and dataframe.iloc[i]['quote_id'] not in G.nodes():
            try:
                G.add_node(dataframe.iloc[i]['quote_id'],text=dataframe.iloc[i]['quote'])
            except:
                G.add_node(dataframe.iloc[i]['quote_id'],text=None)
        
    G.add_edges_from(edges)
    G.remove_node('')
    
    return G

def clean_post(data):
    posts = []
    for post in data:
        try:
            s = soup(post)
            for t in s.find_all('table'):
                t.decompose()
            div = s.find('div')
            posts.append(re.subn('Quote:','',div.text)[0].strip())
        except:
            posts.append('')
    return posts

def get_paths_city_data(dataframe):
    G = make_graph_citydata(dataframe)
    sink_nodes = [node for node, outdegree in dict(G.out_degree(G.nodes())).items() if outdegree == 0]
    source_nodes = [node for node, indegree in dict(G.in_degree(G.nodes())).items() if indegree == 0]
    ss_nodes = [(source, sink) for sink in sink_nodes for source in source_nodes]
    paths = []
    for (source,sink) in ss_nodes:
        for path in nx.all_simple_paths(G, source=source, target=sink):
            paths.append(path)
    return G, paths

def make_thread_embeddings(dataframe, model):
    id_text = {}
    for i in range(len(dataframe)):
        
        id_text[dataframe.iloc[i]['quote_id']] = dataframe.iloc[i]['quote']
        id_text[dataframe.iloc[i]['post_id']] = dataframe.iloc[i]['post']

    G, paths = get_paths_city_data(dataframe)
    chains = []
    for path in paths:
        p = []
        for x in path:
            try:
                p.append(id_text[x])
            except:
                p.append('')
        chains.append(p)
    joint_chains = [' '.join(chain) for chain in chains]
    embeddings = model.encode(joint_chains)
    singletons = [node for node in G.nodes() if node not in flatten_list(paths)]
    singleton_embeddings = model.encode([id_text[s] for s in singletons])
    singleton_texts = [id_text[s] for s in singletons]
    return G, paths, joint_chains, embeddings, singletons, singleton_embeddings,singleton_texts
    
def query_dataframe_quote(df,ie):
    return [df.iloc[i]['cleaned_quote'] for i in range(len(df)) if df.iloc[i]['quote_id'] == ie][0]

def query_dataframe_post(df,ie):
    return [df.iloc[i]['post'] for i in range(len(df)) if df.iloc[i]['post_id'] == ie]

def tfidf_merge(tuplelist):
  d = {}
  for (x,y) in tuplelist:
    d[x] = []
  for (x,y) in tuplelist:
    d[x].append(y)
  return list(zip(d.keys(), [sum(v) for v in d.values()]))

import ast
def reformat_quote_ids(ids):
    qids = []
    for i in ids:
        try:
            qids.append(int(ast.literal_eval(i)[0]))
        except:
            qids.append('')
    return qids

In [7]:
from gensim import models
def gensim_lda(texts, topic_num=5, topic_word_priors=None,numwords=25, eta_=None, tfidf=False):
    """gensim lda wrapper with guided topic modeling"""

    if tfidf != False:
        #process texts
        #build gensim dictionary
        processed_texts = [text_tokenize(text) for text in texts]
        dictionary = gensim.corpora.Dictionary(processed_texts)
    
        #build bag-of-words representation
        bow = [dictionary.doc2bow(text.split()) for text in texts]
        tfidf = models.TfidfModel(bow)
        corpus_tfidf = tfidf[bow]
        #guided lda with eta
        if topic_word_priors and eta_ != None:
            etas = []
        
            for r in range(len(topic_word_priors)):
                eta = []
                for i in range(len(dictionary)):
                    
                    if dictionary[i] in topic_word_priors[r]:
                        eta.append(np.array(eta_))
                    else:
                        eta.append(np.array(1/topic_num))
                etas.append(eta)
    
            model = gensim.models.ldamodel.LdaModel(
                corpus=corpus_tfidf, id2word=dictionary, num_topics=topic_num,
                random_state=42, chunksize=100, eta=np.array(etas),
                eval_every=-1, update_every=1,
                passes=150, alpha='auto', per_word_topics=True)
    
            #transform corpus into topics
            transformed_corpus = model[corpus_tfidf]
    
            #extract topical terms
            topical_terms = [[token for token in m[1].split('"') if not re.findall(r'\d',token)] for m in  model.show_topics(num_words=numwords)]
            return model, topical_terms, transformed_corpus, processed_texts
            
        else:
            #standard lda
            model = gensim.models.ldamodel.LdaModel(
                corpus=corpus_tfidf, id2word=dictionary, num_topics=topic_num,
                random_state=42, chunksize=100, eta=None,
                eval_every=-1, update_every=1,
                passes=150, alpha='auto', per_word_topics=True)
            
            #transform corpus into topics
            transformed_corpus = model[corpus_tfidf]
            topical_terms = [[token for token in m[1].split('"') if not re.findall(r'\d',token)] for m in  model.show_topics(num_words=numwords)]
            return model, topical_terms, transformed_corpus, processed_texts
    else:
        #process texts
        #build gensim dictionary
        processed_texts = [text_tokenize(text) for text in texts]
        dictionary = gensim.corpora.Dictionary(processed_texts)
        
        #build bag-of-words representation
        bow = [dictionary.doc2bow(text.split()) for text in texts]
        #tfidf = models.TfidfModel(bow)
        #corpus_tfidf = tfidf[bow]
    
    
        #guided lda with eta
        if topic_word_priors and eta_ != None:
            etas = []
        
            for r in range(len(topic_word_priors)):
                eta = []
                for i in range(len(dictionary)):
                    
                    if dictionary[i] in topic_word_priors[r]:
                        eta.append(np.array(eta_))
                    else:
                        eta.append(np.array(1/topic_num))
                etas.append(eta)
    
            model = gensim.models.ldamodel.LdaModel(
                corpus=bow, id2word=dictionary, num_topics=topic_num,
                random_state=42, chunksize=100, eta=np.array(etas),
                eval_every=-1, update_every=1,
                passes=150, alpha='auto', per_word_topics=True)
    
            #transform corpus into topics
            transformed_corpus = model[bow]
    
            #extract topical terms
            topical_terms = [[token for token in m[1].split('"') if not re.findall(r'\d',token)] for m in  model.show_topics(num_words=numwords)]
            return model, topical_terms, transformed_corpus, processed_texts
        
        else:
        #standard lda
            model = gensim.models.ldamodel.LdaModel(
                corpus=bow, id2word=dictionary, num_topics=topic_num,
                random_state=42, chunksize=100, eta=None,
                eval_every=-1, update_every=1,
                passes=150, alpha='auto', per_word_topics=True)
            
            #transform corpus into topics
            transformed_corpus = model[bow]
            
            #extract topical terms
            topical_terms = [[token for token in m[1].split('"') if not re.findall(r'\d',token)] for m in  model.show_topics(num_words=25)]
            return model, topical_terms, transformed_corpus, processed_texts
        
        

In [8]:
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans, SpectralClustering, kmeans_plusplus
from sklearn.manifold import SpectralEmbedding
from nltk.corpus import stopwords
import nltk
import re
from scipy.cluster.hierarchy import fclusterdata
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2

stops = stopwords.words('english') + ['said','know','maybe','post','advertisements','advertisement','posted','thread','like','could','should','would','thing']
wn = WordNetLemmatizer()
stemmer = nltk.PorterStemmer()
def text_process(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if not re.findall(r'\.com|___|Advertisements|-|_',token)]
    
    
    return ' '.join([wn.lemmatize(token) for token in tokens if len(token) > 3 and token not in stops and not re.findall(r'[0-9]',token) and not re.findall(r'htt',token)])
def text_tokenize(text):
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    tokens = [token for token in tokens if not re.findall(r'\.com|___|Advertisements|-|_',token)]
    lemmas = [wn.lemmatize(token) for token in tokens if len(token) > 2 and token not in stops]
    filtered = [lemma for lemma in lemmas if not re.findall(r'[0-9]',lemma) and not re.findall(r'htt',lemma)]
    return filtered
    
    
    
# def cluster_thread_embeddings_spectral_tfidf(threads,embeddings, min_words=5, clusters=3,ngrams=(1,1)):
#     sc = SpectralClustering(n_clusters=clusters)
#     sc.fit(embeddings)


#     processed_threads = [text_process(thread) for thread in threads]
#     df = pd.DataFrame({'thread':processed_threads,'label':sc.labels_})
#     Xtrain, xtest, ytrain, ytest = train_test_split(df.thread,df.label,stratify=df.label,test_size=.3)
#     tfidf = TfidfVectorizer(stop_words='english',min_df=min_words,ngram_range=ngrams)
#     tfidf.fit(Xtrain)

#     Xtrain = tfidf.transform(Xtrain)
#     xtest = tfidf.transform(xtest)
#     lg = LogisticRegression(C=1,solver='newton-cg',max_iter=5000)

#     kb = SelectKBest(chi2, k=20).fit(Xtrain, ytrain)
#     X_ = kb.transform(Xtrain)
#     x_test_ = kb.transform(xtest)
#     lg.fit(X_,ytrain)
#     predictions = lg.predict(x_test_)

#     #metrics
#     report_df = pd.DataFrame(classification_report(ytest,predictions,output_dict=True))
#     #print(report_df)
    
#     # lr = LogisticRegression(C=1,solver='newton-cg',max_iter=5000)
#     # lr.fit(tfidf.transform(df.thread),sc.labels_)
#     ff = pd.DataFrame()
#     ff['term'] =tfidf.get_feature_names_out()
#     if clusters > 2:
#         for i in range(clusters):
#             ff['coef_'+str(i)] = tfidf.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])]).toarray().tolist()[0]
       
#     else:
#         ff['coef_0'] = tfidf.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])]).toarray().tolist()[0]
#         ff['coef_1'] = tfidf.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])]).toarray().tolist()[0]
#     ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])

#     topics = get_topical_terms(ff,term_count=25)
#     print(get_topical_coherence(processed_threads, topics,metric='u_mass'))

#     #plot(px.scatter(x=sevecs[:,0],y=sevecs[:,1],color=[str(l) for l in sc.labels_]))
#     return df,ff,tfidf, sc, report_df, topics

def cluster_thread_embeddings_kmeans_tfidf(threads,embeddings, clusters=3,vectorizer='cv',mindf=5, ngrams=(1,1),kbest=5000,init_='k-means++'):
    #if cluster centers provided:
    if type(init_) == np.ndarray:
        print('setting cluster centers')
     
    km = KMeans(n_clusters=clusters,init=init_, random_state=0)
    km.fit(embeddings)
   

    processed_threads = [text_process(thread) for thread in threads]
    df = pd.DataFrame({'thread':processed_threads,'label':km.labels_})

    df_grouped = df.groupby('label')['thread'].apply(list)

    if vectorizer == 'cv':
        cv = CountVectorizer(stop_words='english',min_df=mindf,ngram_range=ngrams)
        #ctfidf = TfidfVectorizer(stop_words='english',min_df=mindf, ngram_range=ngrams)
        CX = cv.fit_transform(df.thread)
        kbc = SelectKBest(chi2, k=kbest).fit(CX, km.labels_)

        ff = pd.DataFrame()
        ff['term'] = np.asarray(cv.get_feature_names_out())[kbc.get_support()]
        if clusters > 2:
            for i in range(clusters):
                ff['coef_'+str(i)] = kbc.transform(cv.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])])).toarray().tolist()[0]
                                                 
        else:
            ff['coef_0'] = kbc.transform(cv.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])])).toarray().tolist()[0]
            ff['coef_1'] = kbc.transform(cv.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])])).toarray().tolist()[0]
            
        ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])
        topics = get_topical_terms(ff,term_count=25)
        print(get_topical_coherence(processed_threads, topics,metric='u_mass'))
        return {'threads':df,'term_weights':ff,'vectorizer':cv, 'clusterer':km, 'topics':topics}
   
    elif vectorizer == 'tfidf':
        tfidf = TfidfVectorizer(stop_words='english',min_df=mindf, ngram_range=ngrams)
        CX = tfidf.fit_transform(df.thread)
        kbc = SelectKBest(chi2, k=kbest).fit(CX, km.labels_)

        ff = pd.DataFrame()
        ff['term'] = np.asarray(tfidf.get_feature_names_out())[kbc.get_support()]
        if clusters > 2:
            for i in range(clusters):
                ff['coef_'+str(i)] = kbc.transform(tfidf.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])])).toarray().tolist()[0]
                                                 
        else:
            ff['coef_0'] = kbc.transform(tfidf.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])])).toarray().tolist()[0]
            ff['coef_1'] = kbc.transform(tfidf.transform([' '.join(df.groupby('label')['thread'].apply(list)[i])])).toarray().tolist()[0]
            
        ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])
        topics = get_topical_terms(ff,term_count=25)
        print(get_topical_coherence(processed_threads, topics,metric='u_mass'))

        return {'threads':df,'term_weights':ff,'vectorizer':tfidf, 'clusterer':km, 'topics':topics}
        
   

# def cluster_thread_embeddings_spectral(threads,embeddings, min_words=5, clusters=3,ngrams=(0,1)):
#     sc = SpectralClustering(n_clusters=clusters)
#     sc.fit(embeddings)


#     processed_threads = [text_process(thread) for thread in threads]
#     df = pd.DataFrame({'thread':processed_threads,'label':sc.labels_})
#     Xtrain, xtest, ytrain, ytest = train_test_split(df.thread,df.label,test_size=.3)
#     tfidf = TfidfVectorizer(stop_words='english',min_df=min_words,ngram_range=ngrams)
#     tfidf.fit(Xtrain)

#     Xtrain = tfidf.transform(Xtrain)
#     xtest = tfidf.transform(xtest)
#     lg = LogisticRegression(C=1,solver='newton-cg',max_iter=5000)
#     lg.fit(Xtrain,ytrain)
#     predictions = lg.predict(xtest)

#     #metrics
#     report_df = pd.DataFrame(classification_report(ytest,predictions,output_dict=True))
#     #print(report_df)
    
#     lr = LogisticRegression(C=1,solver='newton-cg',max_iter=5000)
#     lr.fit(tfidf.transform(df.thread),sc.labels_)
#     ff = pd.DataFrame()
#     ff['term'] =tfidf.get_feature_names_out()
#     if clusters > 2:
#         for i in range(clusters):
#             ff['coef_'+str(i)] = lr.coef_[i]
       
#     else:
#         ff['coef_0'] = lr.coef_[0]
#         ff['coef_1'] = lr.coef_[0] * -1
#     ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])

    
#     se = SpectralEmbedding(n_components=2)
#     sevecs = se.fit_transform(embeddings)

#     #plot(px.scatter(x=sevecs[:,0],y=sevecs[:,1],color=[str(l) for l in sc.labels_]))
#     return df,ff,tfidf, lr, sc, report_df, sevecs
    
# def cluster_thread_embeddings_kmeans(threads,embeddings,clusters=3,ngrams=(1,1)):
#     km = KMeans(n_clusters=clusters)
#     km.fit(embeddings)


#     processed_threads = [text_process(thread) for thread in threads]
#     df = pd.DataFrame({'thread':processed_threads,'label':km.labels_})
    
#     Xtrain, xtest, ytrain, ytest = train_test_split(df.thread.tolist(),df.label.tolist(),test_size=.3)
#     tfidf = TfidfVectorizer(stop_words='english',min_df=5,ngram_range=ngrams)
#     tfidf.fit(Xtrain)
    
#     Xtrain = tfidf.transform(Xtrain)
#     xtest= tfidf.transform(xtest)

#     lg = LogisticRegression(max_iter=5000)
#     lg.fit(Xtrain,ytrain)
#     predictions = lg.predict(xtest)
   
#     report_df = pd.DataFrame(classification_report(ytest,predictions,output_dict=True))
#     #print(report_df)
    

#     lr = LogisticRegression(C=.5,max_iter=5000)
#     lr.fit(tfidf.transform(df.thread.tolist()),df.label.tolist())
#     ff = pd.DataFrame()
#     ff['term'] =tfidf.get_feature_names_out()
#     if clusters > 2:
#         for i in range(clusters):
#             ff['coef_'+str(i)] = lr.coef_[i]
       
        
#     else:
#         ff['coef_0'] = lr.coef_[0]
#         ff['coef_1'] = lr.coef_[0] * -1
        
#     ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])
#     se = SpectralEmbedding(n_components=2)
#     sevecs = se.fit_transform(embeddings)

#     #plot(px.scatter(x=sevecs[:,0],y=sevecs[:,1],color=[str(l) for l in km.labels_]))
#     return df,ff,tfidf, lr, km, report_df, sevecs
# def cluster_thread_embeddings_hdbscan(threads,embeddings,min_cluster_size=5,ngrams=(1,1),metric='manhattan'):
#     hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, gen_min_span_tree=True,metric=metric,allow_single_cluster=False)
#     hdb.fit(embeddings)


#     processed_threads = [text_process(thread) for thread in threads]
#     df = pd.DataFrame({'thread':processed_threads,'label':hdb.labels_})
    
#     Xtrain, xtest, ytrain, ytest = train_test_split(df.thread.tolist(),df.label.tolist(),test_size=.2)
#     tfidf = TfidfVectorizer(stop_words='english',min_df=5,ngram_range=ngrams)
#     tfidf.fit(Xtrain)
    
#     Xtrain = tfidf.transform(Xtrain)
#     xtest= tfidf.transform(xtest)

#     lg = LogisticRegression(max_iter=5000)
#     lg.fit(Xtrain,ytrain)
#     predictions = lg.predict(xtest)
   
#     report_df = pd.DataFrame(classification_report(ytest,predictions,output_dict=True))
#     #print(report_df)
    

#     lr = LogisticRegression(C=.001,max_iter=5000)
#     lr.fit(tfidf.transform(df.thread.tolist()),df.label.tolist())
#     ff = pd.DataFrame()
#     ff['term'] =tfidf.get_feature_names_out()

#     clusters = len(list(set(hdb.labels_)))
#     if clusters > 2:
#         for i in range(clusters):
#             ff['coef_'+str(i)] = lr.coef_[i]
       
#     else:
#         ff['coef_0'] = lr.coef_[0]
#         ff['coef_1'] = lr.coef_[0] * -1

#     ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])
    
#     se = SpectralEmbedding(n_components=2)
#     sevecs = se.fit_transform(embeddings)
#     return df,ff,tfidf, lr, hdb, report_df, sevecs

# def cluster_thread_embeddings(threads, embeddings,cluster_type='spectral', clusters=3):
#     if cluster_type == 'spectral':
#         df, ff, tfidf, lr, sc = cluster_thread_embeddings_spectral(threads, embeddings, clusters)
#         return df, ff, tfidf, lr, sc
#     elif cluster_type == 'kmeans':
#         df, ff, tfidf, lr, km = cluster_thread_embeddings_kmeans(threads, embeddings, clusters)
#         return df, ff, tfidf, lr, km

def fcluster_thread_embeddings(threads,embeddings,threshold=2):
    
    processed_threads = [text_process(thread) for thread in threads]

    
    ls = fclusterdata(embeddings,t=threshold,criterion='distance',method='ward')
        
    df = pd.DataFrame({'thread':processed_threads,'label':ls})
    Xtrain, xtest, ytrain, ytest = train_test_split(df.thread,df.label,test_size=.2)
    tfidf = TfidfVectorizer(stop_words='english',min_df=5,ngram_range=(1,3))
    
    lg = LogisticRegression()
    lg.fit(tfidf.fit_transform(Xtrain),ytrain)
    predictions = lg.predict(tfidf.transform(xtest))
    print(classification_report(ytest,predictions))
    
    lr = LogisticRegression()
    lr.fit(tfidf.transform(df.thread),ls)
    ff = pd.DataFrame()
    ff['term'] =tfidf.get_feature_names_out()
    for i in range(len(Counter(ls))):
        ff['coef_'+str(i)] = lr.coef_[i]
       
    ff = pd.DataFrame([ff.iloc[i] for i in range(len(ff)) if len(ff.iloc[i]['term']) > 3])
    
    # se = SpectralEmbedding(n_components=2)
    # sevecs = se.fit_transform(embeddings)
    
    #plot(px.scatter(x=sevecs[:,0],y=sevecs[:,1],color=[str(l) for l in ls]))

    return df, ff

#corpus_file = api.load('crawl-300d-2M-subword/crawl-300d-2M-subword.bin')

# with open('crawl-300d-2M-subword/crawl-300d-2M-subword.bin', 'rb') as fin:
#     fast = gensim.models.fasttext.load_facebook_model(fin)

In [23]:
fast = gensim.models.fasttext.load_facebook_model('crawl-300d-2M-subword/crawl-300d-2M-subword.bin')

In [ ]:
import gensim
with open('crawl-300d-2M-subword/crawl-300d-2M-subword.bin', 'rb') as fin:
    fast = gensim.models.fasttext.load_facebook_model(fin)

In [12]:
import numpy as np

from scipy.spatial import distance
from itertools import combinations


def proportion_unique_words(topics, topk=10):
    """
    compute the proportion of unique words

    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity will be computed
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than '+str(topk))
    else:
        unique_words = set()
        for topic in topics:
            unique_words = unique_words.union(set(topic[:topk]))
        puw = len(unique_words) / (topk * len(topics))
        return puw


def irbo(topics, weight=0.9, topk=10):
    """
    compute the inverted rank-biased overlap

    Parameters
    ----------
    topics: a list of lists of words
    weight: p (float), default 1.0: Weight of each
        agreement at depth d:p**(d-1). When set
        to 1.0, there is no weight, the rbo returns
        to average overlap.
    topk: top k words on which the topic diversity
          will be computed

    Returns
    -------
    irbo : score of the rank biased overlap over the topics
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        collect = []
        for list1, list2 in combinations(topics, 2):
            word2index = get_word2index(list1, list2)
            indexed_list1 = [word2index[word] for word in list1]
            indexed_list2 = [word2index[word] for word in list2]
            rbo_val = rbo(indexed_list1[:topk], indexed_list2[:topk], p=weight)[2]
            collect.append(rbo_val)
        return 1 - np.mean(collect)


def word_embedding_irbo(topics, word_embedding_model, weight=0.9, topk=10):
    '''
    compute the word embedding-based inverted rank-biased overlap

    Parameters
    ----------
    topics: a list of lists of words
    weight: p (float), default 1.0: Weight of each agreement at depth d:
    p**(d-1). When set to 1.0, there is no weight, the rbo returns to average overlap.
    
    Returns
    -------
    weirbo: word embedding-based inverted rank_biased_overlap over the topics
    '''
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        collect = []
        for list1, list2 in combinations(topics, 2):
            word2index = get_word2index(list1, list2)
            index2word = {v: k for k, v in word2index.items()}
            indexed_list1 = [word2index[word] for word in list1]
            indexed_list2 = [word2index[word] for word in list2]
            rbo_val = word_embeddings_rbo(indexed_list1[:topk], indexed_list2[:topk], p=weight,
                                          index2word=index2word, word2vec=word_embedding_model)[2]
            collect.append(rbo_val)
        return 1 - np.mean(collect)


def pairwise_jaccard_diversity(topics, topk=10):
    '''
    compute the average pairwise jaccard distance between the topics 
  
    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity
          will be computed
    
    Returns
    -------
    pjd: average pairwise jaccard distance
    '''
    dist = 0
    count = 0
    for list1, list2 in combinations(topics, 2):
        js = 1 - len(set(list1).intersection(set(list2)))/len(set(list1).union(set(list2)))
        dist = dist + js
        count = count + 1
    return dist/count


def pairwise_word_embedding_distance(topics, word_embedding_model, topk=10):
    """
    :param topk: how many most likely words to consider in the evaluation
    :return: topic coherence computed on the word embeddings similarities
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        count = 0
        sum_dist = 0
        for list1, list2 in combinations(topics, 2):
            count = count+1
            word_counts = 0
            dist = 0
            for word1 in list1[:topk]:
                for word2 in list2[:topk]:
                    dist = dist + distance.cosine(word_embedding_model.wv[word1], word_embedding_model.wv[word2])
                    word_counts = word_counts + 1

            dist = dist/word_counts
            sum_dist = sum_dist + dist
        return sum_dist/count


def centroid_distance(topics, word_embedding_model, topk=10):
    """
    :param topk: how many most likely words to consider in the evaluation
    :return: topic coherence computed on the word embeddings similarities
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than topk')
    else:
        count = 0
        for list1, list2 in combinations(topics, 2):
            count = count + 1
            centroid1 = np.zeros(word_embedding_model.vector_size)
            centroid2 = np.zeros(word_embedding_model.vector_size)
            for word1 in list1[:topk]:
                centroid1 = centroid1 + word_embedding_model[word1]
            for word2 in list2[:topk]:
                centroid2 = centroid2 + word_embedding_model[word2]
            centroid1 = centroid1 / len(list1[:topk])
            centroid2 = centroid2 / len(list2[:topk])
        return distance.cosine(centroid1, centroid2)


def get_word2index(list1, list2):
    words = set(list1)
    words = words.union(set(list2))
    word2index = {w: i for i, w in enumerate(words)}
    return word2index
"""Rank-biased overlap, a ragged sorted list similarity measure.
See http://doi.acm.org/10.1145/1852102.1852106 for details. All functions
directly corresponding to concepts from the paper are named so that they can be
clearly cross-identified.
The definition of overlap has been modified to account for ties. Without this,
results for lists with tied items were being inflated. The modification itself
is not mentioned in the paper but seems to be reasonable, see function
``overlap()``. Places in the code which diverge from the spec in the paper
because of this are highlighted with comments.
The two main functions for performing an RBO analysis are ``rbo()`` and
``rbo_dict()``; see their respective docstrings for how to use them.
The following doctest just checks that equivalent specifications of a
problem yield the same result using both functions:
    >>> lst1 = [{"c", "a"}, "b", "d"]
    >>> lst2 = ["a", {"c", "b"}, "d"]
    >>> ans_rbo = _round(rbo(lst1, lst2, p=.9))
    >>> dct1 = dict(a=1, b=2, c=1, d=3)
    >>> dct2 = dict(a=1, b=2, c=2, d=3)
    >>> ans_rbo_dict = _round(rbo_dict(dct1, dct2, p=.9, sort_ascending=True))
    >>> ans_rbo == ans_rbo_dict
    True
"""


import math
from bisect import bisect_left
from collections import namedtuple


RBO = namedtuple("RBO", "min res ext")
RBO.__doc__ += ": Result of full RBO analysis"
RBO.min.__doc__ = "Lower bound estimate"
RBO.res.__doc__ = "Residual corresponding to min; min + res is an upper bound estimate"
RBO.ext.__doc__ = "Extrapolated point estimate"


def _round(obj):
    if isinstance(obj, RBO):
        return RBO(_round(obj.min), _round(obj.res), _round(obj.ext))
    else:
        return round(obj, 3)


def set_at_depth(lst, depth):
    ans = set()
    for v in lst[:depth]:
        if isinstance(v, set):
            ans.update(v)
        else:
            ans.add(v)
    return ans


def raw_overlap(list1, list2, depth):
    """Overlap as defined in the article.
    """
    set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    return len(set1.intersection(set2)), len(set1), len(set2)


def overlap(list1, list2, depth):
    """Overlap which accounts for possible ties.
    This isn't mentioned in the paper but should be used in the ``rbo*()``
    functions below, otherwise overlap at a given depth might be > depth which
    inflates the result.
    There are no guidelines in the paper as to what's a good way to calculate
    this, but a good guess is agreement scaled by the minimum between the
    requested depth and the lengths of the considered lists (overlap shouldn't
    be larger than the number of ranks in the shorter list, otherwise results
    are conspicuously wrong when the lists are of unequal lengths -- rbo_ext is
    not between rbo_min and rbo_min + rbo_res.
    >>> overlap("abcd", "abcd", 3)
    3.0
    >>> overlap("abcd", "abcd", 5)
    4.0
    >>> overlap(["a", {"b", "c"}, "d"], ["a", {"b", "c"}, "d"], 2)
    2.0
    >>> overlap(["a", {"b", "c"}, "d"], ["a", {"b", "c"}, "d"], 3)
    3.0
    """
    ov = agreement(list1, list2, depth) * min(depth, len(list1), len(list2))
    return ov
    # NOTE: comment the preceding and uncomment the following line if you want
    # to stick to the algorithm as defined by the paper
    # return raw_overlap(list1, list2, depth)[0]


def agreement(list1, list2, depth):
    """Proportion of shared values between two sorted lists at given depth.
    >>> _round(agreement("abcde", "abdcf", 1))
    1.0
    >>> _round(agreement("abcde", "abdcf", 3))
    0.667
    >>> _round(agreement("abcde", "abdcf", 4))
    1.0
    >>> _round(agreement("abcde", "abdcf", 5))
    0.8
    >>> _round(agreement([{1, 2}, 3], [1, {2, 3}], 1))
    0.667
    >>> _round(agreement([{1, 2}, 3], [1, {2, 3}], 2))
    1.0
    """
    len_intersection, len_set1, len_set2 = raw_overlap(list1, list2, depth)
    return 2 * len_intersection / (len_set1 + len_set2)


def cumulative_agreement(list1, list2, depth):
    return (agreement(list1, list2, d) for d in range(1, depth + 1))


def average_overlap(list1, list2, depth=None):
    """Calculate average overlap between ``list1`` and ``list2``.
    >>> _round(average_overlap("abcdefg", "zcavwxy", 1))
    0.0
    >>> _round(average_overlap("abcdefg", "zcavwxy", 2))
    0.0
    >>> _round(average_overlap("abcdefg", "zcavwxy", 3))
    0.222
    >>> _round(average_overlap("abcdefg", "zcavwxy", 4))
    0.292
    >>> _round(average_overlap("abcdefg", "zcavwxy", 5))
    0.313
    >>> _round(average_overlap("abcdefg", "zcavwxy", 6))
    0.317
    >>> _round(average_overlap("abcdefg", "zcavwxy", 7))
    0.312
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    return sum(cumulative_agreement(list1, list2, depth)) / depth


def rbo_at_k(list1, list2, p, depth=None):
    # ``p**d`` here instead of ``p**(d - 1)`` because enumerate starts at
    # 0
    depth = min(len(list1), len(list2)) if depth is None else depth
    d_a = enumerate(cumulative_agreement(list1, list2, depth))
    return (1 - p) * sum(p ** d * a for (d, a) in d_a)


def rbo_min(list1, list2, p, depth=None):
    """Tight lower bound on RBO.
    See equation (11) in paper.
    >>> _round(rbo_min("abcdefg", "abcdefg", .9))
    0.767
    >>> _round(rbo_min("abcdefgh", "abcdefg", .9))
    0.767
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    x_k = overlap(list1, list2, depth)
    log_term = x_k * math.log(1 - p)
    sum_term = sum(
        p ** d / d * (overlap(list1, list2, d) - x_k) for d in range(1, depth + 1)
    )
    return (1 - p) / p * (sum_term - log_term)


def rbo_res(list1, list2, p):
    """Upper bound on residual overlap beyond evaluated depth.
    See equation (30) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible. In particular, for identical lists, ``rbo_min()`` and
    ``rbo_res()`` should add up to 1, which is the case.
    >>> _round(rbo_res("abcdefg", "abcdefg", .9))
    0.233
    >>> _round(rbo_res("abcdefg", "abcdefghijklmnopqrstuvwxyz", .9))
    0.239
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l)
    # since overlap(...) can be fractional in the general case of ties and f
    # must be an integer --> math.ceil()
    f = int(math.ceil(l + s - x_l))
    # upper bound of range() is non-inclusive, therefore + 1 is needed
    term1 = s * sum(p ** d / d for d in range(s + 1, f + 1))
    term2 = l * sum(p ** d / d for d in range(l + 1, f + 1))
    term3 = x_l * (math.log(1 / (1 - p)) - sum(p ** d / d for d in range(1, f + 1)))
    return p ** s + p ** l - p ** f - (1 - p) / p * (term1 + term2 + term3)


def rbo_ext(list1, list2, p):
    """RBO point estimate based on extrapolating observed overlap.
    See equation (32) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible.
    >>> _round(rbo_ext("abcdefg", "abcdefg", .9))
    1.0
    >>> _round(rbo_ext("abcdefg", "bacdefg", .9))
    0.9
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l)
    x_s = overlap(list1, list2, s)
    # the paper says overlap(..., d) / d, but it should be replaced by
    # agreement(..., d) defined as per equation (28) so that ties are handled
    # properly (otherwise values > 1 will be returned)
    # sum1 = sum(p**d * overlap(list1, list2, d)[0] / d for d in range(1, l + 1))
    sum1 = sum(p ** d * agreement(list1, list2, d) for d in range(1, l + 1))
    sum2 = sum(p ** d * x_s * (d - s) / s / d for d in range(s + 1, l + 1))
    term1 = (1 - p) / p * (sum1 + sum2)
    term2 = p ** l * ((x_l - x_s) / l + x_s / s)
    return term1 + term2


def rbo(list1, list2, p):
    """Complete RBO analysis (lower bound, residual, point estimate).
    ``list`` arguments should be already correctly sorted iterables and each
    item should either be an atomic value or a set of values tied for that
    rank. ``p`` is the probability of looking for overlap at rank k + 1 after
    having examined rank k.
    >>> lst1 = [{"c", "a"}, "b", "d"]
    >>> lst2 = ["a", {"c", "b"}, "d"]
    >>> _round(rbo(lst1, lst2, p=.9))
    RBO(min=0.489, res=0.477, ext=0.967)
    """
    if not 0 <= p <= 1:
        raise ValueError("The ``p`` parameter must be between 0 and 1.")
    args = (list1, list2, p)
    return RBO(rbo_min(*args), rbo_res(*args), rbo_ext(*args))


def sort_dict(dct, *, ascending=False):
    """Sort keys in ``dct`` according to their corresponding values.
    Sorts in descending order by default, because the values are
    typically scores, i.e. the higher the better. Specify
    ``ascending=True`` if the values are ranks, or some sort of score
    where lower values are better.
    Ties are handled by creating sets of tied keys at the given position
    in the sorted list.
    >>> dct = dict(a=1, b=2, c=1, d=3)
    >>> list(sort_dict(dct)) == ['d', 'b', {'a', 'c'}]
    True
    >>> list(sort_dict(dct, ascending=True)) == [{'a', 'c'}, 'b', 'd']
    True
    """
    scores = []
    items = []
    # items should be unique, scores don't have to
    for item, score in dct.items():
        if not ascending:
            score *= -1
        i = bisect_left(scores, score)
        if i == len(scores):
            scores.append(score)
            items.append(item)
        elif scores[i] == score:
            existing_item = items[i]
            if isinstance(existing_item, set):
                existing_item.add(item)
            else:
                items[i] = {existing_item, item}
        else:
            scores.insert(i, score)
            items.insert(i, item)
    return items


def rbo_dict(dict1, dict2, p, *, sort_ascending=False):
    """Wrapper around ``rbo()`` for dict input.
    Each dict maps items to be sorted to the score according to which
    they should be sorted. The RBO analysis is then performed on the
    resulting sorted lists.
    The sort is descending by default, because scores are typically the
    higher the better, but this can be overridden by specifying
    ``sort_ascending=True``.
    >>> dct1 = dict(a=1, b=2, c=1, d=3)
    >>> dct2 = dict(a=1, b=2, c=2, d=3)
    >>> _round(rbo_dict(dct1, dct2, p=.9, sort_ascending=True))
    RBO(min=0.489, res=0.477, ext=0.967)
    """
    list1, list2 = (
        sort_dict(dict1, ascending=sort_ascending),
        sort_dict(dict2, ascending=sort_ascending),
    )
    return rbo(list1, list2, p)


if __name__ in ("__main__", "__console__"):
    import doctest

    doctest.testmod()

from __future__ import division

import math
from bisect import bisect_left
from collections import namedtuple
from collections import OrderedDict

RBO = namedtuple("RBO", "min res ext")
RBO.__doc__ += ": Result of full RBO analysis"
RBO.min.__doc__ = "Lower bound estimate"
RBO.res.__doc__ = "Residual corresponding to min; min + res is an upper bound estimate"
RBO.ext.__doc__ = "Extrapolated point estimate"

def _round(obj):
    if isinstance(obj, RBO):
        return RBO(_round(obj.min), _round(obj.res), _round(obj.ext))
    else:
        return round(obj, 3)


def set_at_depth(lst, depth):
    ans = set()
    for v in lst[:depth]:
        if isinstance(v, set):
            ans.update(v)
        else:
            ans.add(v)
    return ans


def embeddings_overlap(list1, list2, depth, index2word, word2vec):
    #set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    #return len(set1.intersection(set2)), len(set1), len(set2)

    set1, set2 = set_at_depth(list1, depth), set_at_depth(list2, depth)
    word_list1 = [index2word[index] for index in list1]
    word_list2 = [index2word[index] for index in list2]

    similarities = {}
    for w1 in word_list1[:depth]:
        for w2 in word_list2[:depth]:
            similarities[(w1,w2)] = word2vec.similarity(w1, w2)

    similarities = OrderedDict(sorted(similarities.items(), key=lambda x: -x[1]))

    e_ov = 0
    key_list = list(similarities.keys())
    for k in key_list:
        if k in similarities.keys():
            #print(k, similarities[k])
            e_ov = e_ov + similarities[k]
            similarities = {save_k: v for save_k, v in similarities.items()
                            if save_k[0] != k[0] and save_k[1] != k[1]}
    #e_ov = 1
    #print("****")
    return e_ov, len(set1), len(set2)


def overlap(list1, list2, depth, index2word, word2vec):
    #return agreement(list1, list2, depth) * min(depth, len(list1), len(list2))
    # NOTE: comment the preceding and uncomment the following line if you want
    # to stick to the algorithm as defined by the paper
    ov = embeddings_overlap(list1, list2, depth, index2word, word2vec)[0]
    return ov


def agreement(list1, list2, depth, index2word, word2vec):
    """Proportion of shared values between two sorted lists at given depth."""
    len_intersection, len_set1, len_set2 = embeddings_overlap(list1, list2, depth, index2word, word2vec)
    return 2 * len_intersection / (len_set1 + len_set2)


def cumulative_agreement(list1, list2, depth, index2word, word2vec):
    return (agreement(list1, list2, d, index2word, word2vec) for d in range(1, depth + 1))


def average_overlap(list1, list2, index2word, word2vec, depth=None):
    """Calculate average overlap between ``list1`` and ``list2``.
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    return sum(cumulative_agreement(list1, list2, depth, index2word=index2word, word2vec=word2vec)) / depth


def rbo_at_k(list1, list2, p, index2word, word2vec, depth=None):
    # ``p**d`` here instead of ``p**(d - 1)`` because enumerate starts at
    # 0
    depth = min(len(list1), len(list2)) if depth is None else depth
    d_a = enumerate(cumulative_agreement(list1, list2, depth, index2word=index2word, word2vec=word2vec))
    return (1 - p) * sum(p ** d * a for (d, a) in d_a)


def rbo_min(list1, list2, p, index2word, word2vec, depth=None):
    """Tight lower bound on RBO.
    See equation (11) in paper.
    """
    depth = min(len(list1), len(list2)) if depth is None else depth
    x_k = overlap(list1, list2, depth, index2word, word2vec)
    log_term = x_k * math.log(1 - p)
    sum_term = sum(
        p ** d / d * (overlap(list1, list2, d, index2word, word2vec=word2vec) - x_k) for d in range(1, depth + 1)
    )
    return (1 - p) / p * (sum_term - log_term)


def rbo_res(list1, list2, p, index2word, word2vec):
    """Upper bound on residual overlap beyond evaluated depth.
    See equation (30) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible. In particular, for identical lists, ``rbo_min()`` and
    ``rbo_res()`` should add up to 1, which is the case.
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l, index2word, word2vec)
    # since overlap(...) can be fractional in the general case of ties and f
    # must be an integer --> math.ceil()
    f = int(math.ceil(l + s - x_l))
    # upper bound of range() is non-inclusive, therefore + 1 is needed
    term1 = s * sum(p ** d / d for d in range(s + 1, f + 1))
    term2 = l * sum(p ** d / d for d in range(l + 1, f + 1))
    term3 = x_l * (math.log(1 / (1 - p)) - sum(p ** d / d for d in range(1, f + 1)))
    return p ** s + p ** l - p ** f - (1 - p) / p * (term1 + term2 + term3)


def rbo_ext(list1, list2, p, index2word, word2vec):
    """RBO point estimate based on extrapolating observed overlap.
    See equation (32) in paper.
    NOTE: The doctests weren't verified against manual computations but seem
    plausible.
    >>> _round(rbo_ext("abcdefg", "abcdefg", .9))
    1.0
    >>> _round(rbo_ext("abcdefg", "bacdefg", .9))
    0.9
    """
    S, L = sorted((list1, list2), key=len)
    s, l = len(S), len(L)
    x_l = overlap(list1, list2, l, index2word, word2vec)
    x_s = overlap(list1, list2, s, index2word, word2vec)
    # the paper says overlap(..., d) / d, but it should be replaced by
    # agreement(..., d) defined as per equation (28) so that ties are handled
    # properly (otherwise values > 1 will be returned)
    # sum1 = sum(p**d * overlap(list1, list2, d)[0] / d for d in range(1, l + 1))
    sum1 = sum(p ** d * agreement(list1, list2, d, index2word=index2word, word2vec=word2vec)
               for d in range(1, l + 1))
    sum2 = sum(p ** d * x_s * (d - s) / s / d for d in range(s + 1, l + 1))
    term1 = (1 - p) / p * (sum1 + sum2)
    term2 = p ** l * ((x_l - x_s) / l + x_s / s)
    return term1 + term2


def word_embeddings_rbo(list1, list2, p, index2word, word2vec):
    """Complete RBO analysis (lower bound, residual, point estimate).
    ``list`` arguments should be already correctly sorted iterables and each
    item should either be an atomic value or a set of values tied for that
    rank. ``p`` is the probability of looking for overlap at rank k + 1 after
    having examined rank k.
    >>> lst1 = [{"c", "a"}, "b", "d"]
    >>> lst2 = ["a", {"c", "b"}, "d"]
    >>> _round(rbo(lst1, lst2, p=.9))
    RBO(min=0.489, res=0.477, ext=0.967)
    """
    if not 0 <= p <= 1:
        raise ValueError("The ``p`` parameter must be between 0 and 1.")
    args = (list1, list2, p, index2word, word2vec)

    return RBO(rbo_min(*args), rbo_res(*args), rbo_ext(*args))


def sort_dict(dct, *, ascending=False):
    """Sort keys in ``dct`` according to their corresponding values.
    Sorts in descending order by default, because the values are
    typically scores, i.e. the higher the better. Specify
    ``ascending=True`` if the values are ranks, or some sort of score
    where lower values are better.
    Ties are handled by creating sets of tied keys at the given position
    in the sorted list.
    >>> dct = dict(a=1, b=2, c=1, d=3)
    >>> list(sort_dict(dct)) == ['d', 'b', {'a', 'c'}]
    True
    >>> list(sort_dict(dct, ascending=True)) == [{'a', 'c'}, 'b', 'd']
    True
    """
    scores = []
    items = []
    # items should be unique, scores don't have to
    for item, score in dct.items():
        if not ascending:
            score *= -1
        i = bisect_left(scores, score)
        if i == len(scores):
            scores.append(score)
            items.append(item)
        elif scores[i] == score:
            existing_item = items[i]
            if isinstance(existing_item, set):
                existing_item.add(item)
            else:
                items[i] = {existing_item, item}
        else:
            scores.insert(i, score)
            items.insert(i, item)
    return items


def rbo_dict(dict1, dict2, p, index2word, word2vec, *, sort_ascending=False):
    """Wrapper around ``rbo()`` for dict input.
    Each dict maps items to be sorted to the score according to which
    they should be sorted. The RBO analysis is then performed on the
    resulting sorted lists.
    The sort is descending by default, because scores are typically the
    higher the better, but this can be overridden by specifying
    ``sort_ascending=True``.
    """
    list1, list2 = (
        sort_dict(dict1, ascending=sort_ascending),
        sort_dict(dict2, ascending=sort_ascending),
    )
    return word_embeddings_rbo(list1, list2, p, index2word, word2vec)

def get_diversity_scores(topics, model, topn=10, topic_type=''):
    df = pd.DataFrame({"puw:":proportion_unique_words(topics, topk=topn),
"jd:": pairwise_jaccard_diversity(topics, topk=topn),
"we-pd:": pairwise_word_embedding_distance(topics, model, topk=topn),
"we-cd:": centroid_distance(topics, model.wv, topk=topn),
"we-irbo p=0.5:":word_embedding_irbo(topics,model.wv, weight=0.5, topk=topn),
"we-irbo p=0.9:":word_embedding_irbo(topics,model.wv, weight=0.9, topk=topn)},index=[topic_type])
    return df


SyntaxError: from __future__ imports must occur at the beginning of the file (2635683237.py, line 186)

In [9]:
citydata_df = pd.read_parquet('citydata.parquet')

In [10]:
covid = pd.read_parquet('citydata-covid.parquet')
crime = pd.read_parquet('citydata-crime.parquet')
metro = pd.read_parquet('citydata-metro.parquet')
plan = pd.read_parquet('citydata-2035plan.parquet')
retail = pd.read_parquet('citydata-retail.parquet')

In [11]:
CO = make_graph_citydata(covid)
CR = make_graph_citydata(crime)
ME = make_graph_citydata(metro)
PL = make_graph_citydata(plan)
R = make_graph_citydata(retail)

In [ ]:
citydata_df

In [39]:
first_posts = [covid['post'].map(lambda x: re.subn(r'Advertisements','',x)[0])[0],
crime['post'].map(lambda x: re.subn(r'Advertisements','',x)[0])[0] + ' ' + metro['post'].map(lambda x: re.subn(r'Advertisements','',x)[0])[0],
plan['post'].map(lambda x: re.subn(r'Advertisements','',x)[0])[0],
retail['post'].map(lambda x: re.subn(r'Advertisements','',x)[0])[0],]

#first_posts_embeddings = smodel.encode(first_posts)


In [33]:
covid_labels = ['covid' for i in range(len(covid))]
crime_labels = ['covid' for i in range(len(crime))]
metro_labels = ['covid' for i in range(len(metro))]
plan_labels = ['covid' for i in range(len(plan))]
retail_labels = ['covid' for i in range(len(retail))]

In [17]:
CITY, city_paths, city_chains, city_embeddings, city_singletons,city_singleton_embeddings, city_singleton_texts = make_thread_embeddings(citydata_df,smodel)



In [18]:
save_pickle([CITY, city_paths, city_chains, city_embeddings, city_singletons,city_singleton_embeddings, city_singleton_texts],'citydata_processed.pkl')

In [12]:
city = open_pickle('citydata_processed.pkl')


In [13]:
CITY = city[0]
city_paths = city[1]
city_chains = city[2]
city_embeddings = city[3]
city_singletons = city[4]
city_singleton_embeddings = city[5]
city_singleton_texts = city[6]

## BERTopic

In [40]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from hdbscan import HDBSCAN

# seed_topic_list = [["drug", "cancer", "drugs", "doctor"],
#                    ["windows", "drive", "dos", "file"],
#                    ["space", "launch", "orbit", "lunar"]]



# Train BERTopic with a custom CountVectorizer
vectorizer_model = CountVectorizer(min_df=3, stop_words='english')
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model = BERTopic(hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model,nr_topics=5, top_n_words=25)

topics, probs = topic_model.fit_transform(threads)

tw = [[w for (w,s) in topic_model.get_topic(-1)],
      [w for (w,s) in topic_model.get_topic(0)], 
      [w for (w,s) in topic_model.get_topic(1)],  
      [w for (w,s) in topic_model.get_topic(2)],
     [w for (w,s) in topic_model.get_topic(3)],]
#print(get_topical_coherence(threads, tw,metric='u_mass'))
print(get_coherence_diversity_scores(threads, tw,fast,topn=25,topic_type='BERTopic Threads'))

C:\Users\rmomi\anaconda3\envs\bertopic\lib\site-packages\scipy\spatial\distance.py:622: RuntimeWarning:

invalid value encountered in scalar divide



                  u_mass_coherence   puw:       jd:    we-pd:   we-cd:  \
BERTopic Threads         -5.591325  0.496  0.795484  0.462137  0.28213   

                  we-irbo p=0.5:  we-irbo p=0.9:  
BERTopic Threads        0.624762         0.53839  


In [46]:

tw = [[w for (w,s) in topic_model.get_topic(-1)],
      [w for (w,s) in topic_model.get_topic(0)], 
      [w for (w,s) in topic_model.get_topic(1)],  
      [w for (w,s) in topic_model.get_topic(2)],
     [w for (w,s) in topic_model.get_topic(3)],]

pd.DataFrame(tw)[[0,1,2,3,4,5,6,7,8,9]]

,0,1,2,3,4,5,6,7,8,9
0,city,like,people,philly,just,think,dont,philadelphia,new,street
1,city,like,new,store,think,just,philadelphia,retail,stores,people
2,people,crime,city,dont,just,like,white,im,know,year
3,hmmm,hello,fancy,update,haha,,,,,
4,inga,like,just,news,article,dont,read,writing,people,does


In [45]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4639,-1_city_like_people_philly,"[city, like, people, philly, just, think, dont...","[ What makes a city world class: History, tour..."
1,0,6363,0_city_like_new_store,"[city, like, new, store, think, just, philadel...",[ The Country Club Plaza is generally consider...
2,1,2804,1_people_crime_city_dont,"[people, crime, city, dont, just, like, white,...",[ What is going on with this forum and the may...
3,2,701,2_hmmm_hello_fancy_update,"[hmmm, hello, fancy, update, haha, , , , , , ,...","[ haha , Update., Fancy]"
4,3,501,3_inga_like_just_news,"[inga, like, just, news, article, dont, read, ...","[ She's the ""architecture"" critic for the Inqu..."


In [39]:
post_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,10408,-1_city_like_people_just,"[city, like, people, just, philly, philadelphi...","[ In my opinion, you want to turn Philadelphia..."
1,0,8720,0_city_like_think_new,"[city, like, think, new, just, people, buildin...","[ i am not trying to be a hater, but that is U..."
2,1,2591,1_white_people_city_dont,"[white, people, city, dont, like, crime, im, y...",[ I definitely agree that Millennials have to ...
3,2,1028,2_like_inga_news_just,"[like, inga, news, just, dont, people, article...","[ In her typical hysterical fashion, Inga is a..."
4,3,705,3_bau_hello_nasty_bart,"[bau, hello, nasty, bart, fancy, update, haha,...","[ Fancy, Nasty., bau?]"


In [38]:
# Train BERTopic with a custom CountVectorizer
post_vectorizer_model = CountVectorizer(min_df=1, stop_words='english')
post_hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
post_topic_model = BERTopic(hdbscan_model=post_hdbscan_model, vectorizer_model=post_vectorizer_model,nr_topics=5,top_n_words=25)

post_topics, post_probs = post_topic_model.fit_transform(posts_quotes)

tw = [[w for (w,s) in post_topic_model.get_topic(-1)],
      [w for (w,s) in post_topic_model.get_topic(0)], 
      [w for (w,s) in post_topic_model.get_topic(1)],  
      [w for (w,s) in post_topic_model.get_topic(2)],
     [w for (w,s) in post_topic_model.get_topic(3)],]
#print(get_topical_coherence(posts_quotes, tw,metric='u_mass'))
print(get_coherence_diversity_scores(threads, tw,fast,topn=25,topic_type='BERTopic Posts'))

C:\Users\rmomi\anaconda3\envs\bertopic\lib\site-packages\scipy\spatial\distance.py:622: RuntimeWarning:

invalid value encountered in scalar divide



                u_mass_coherence  puw:       jd:    we-pd:    we-cd:  \
BERTopic Posts         -5.121572  0.52  0.780486  0.522732  0.231391   

                we-irbo p=0.5:  we-irbo p=0.9:  
BERTopic Posts        0.587038        0.495225  


In [42]:
tw = [[w for (w,s) in post_topic_model.get_topic(-1)],
      [w for (w,s) in post_topic_model.get_topic(0)], 
      [w for (w,s) in post_topic_model.get_topic(1)],  
      [w for (w,s) in post_topic_model.get_topic(2)],
     [w for (w,s) in post_topic_model.get_topic(3)],]

In [43]:
pd.DataFrame(tw)[[0,1,2,3,4,5,6,7,8,9]]

,0,1,2,3,4,5,6,7,8,9
0,city,like,people,just,philly,philadelphia,think,dont,street,center
1,city,like,think,new,just,people,building,philadelphia,dont,philly
2,white,people,city,dont,like,crime,im,year,just,murders
3,like,inga,news,just,dont,people,article,think,does,thread
4,bau,hello,nasty,bart,fancy,update,haha,awful,ok,finally


In [16]:
covid_nodes = [(text_process(CO.nodes()[k]['text']).split(),k) for (k,v) in sorted(dict(CO.degree()).items(), key=lambda x:x[1],reverse=True)][0]
crime_nodes = [(text_process(CR.nodes()[k]['text']).split(),k) for (k,v) in sorted(dict(CR.degree()).items(), key=lambda x:x[1],reverse=True)][0]
metro_nodes = [(text_process(ME.nodes()[k]['text']).split(),k) for (k,v) in sorted(dict(ME.degree()).items(), key=lambda x:x[1],reverse=True)][0]
plan_nodes = [(text_process(PL.nodes()[k]['text']).split(),k) for (k,v) in sorted(dict(PL.degree()).items(), key=lambda x:x[1],reverse=True)][0]
retail_nodes = [(text_process(R.nodes()[k]['text']).split(),k) for (k,v) in sorted(dict(R.degree()).items(), key=lambda x:x[1],reverse=True)][0]


#degree_seed_embeddings = smodel.encode([covid_nodes,crime_nodes + metro_nodes, plan_nodes, retail_nodes])
degree_seeds = [covid_nodes,crime_nodes,metro_nodes, plan_nodes, retail_nodes]



In [ ]:
import numpy as np
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from hdbscan import HDBSCAN

# seed_topic_list = [d[0][:10] for d in degree_seeds]


seed_topic_list = [["drug", "cancer", "drugs", "doctor"],
                   ["windows", "drive", "dos", "file"],
                   ["space", "launch", "orbit", "lunar"]]

degree_vectorizer_model = CountVectorizer(min_df=1, stop_words='english')
degree_hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
degree_topic_model = BERTopic(seed_topic_list=seed_topic_list,hdbscan_model=degree_hdbscan_model, vectorizer_model=degree_vectorizer_model,nr_topics=5,top_n_words=25)

degree_topics, degree_probs = degree_topic_model.fit_transform(posts_quotes)

tw = [[w for (w,s) in post_topic_model.get_topic(-1)],
      [w for (w,s) in post_topic_model.get_topic(0)], 
      [w for (w,s) in post_topic_model.get_topic(1)],  
      [w for (w,s) in post_topic_model.get_topic(2)],
     [w for (w,s) in post_topic_model.get_topic(3)],]
print(get_topical_coherence(posts_quotes, tw,metric='u_mass'))

## Thread Topic Modeling

In [14]:
#city_embeddings = np.concatenate([city_embeddings, city_singleton_embeddings])
threads = city_chains + city_singleton_texts

In [20]:
processed_threads = [text_process(thread) for thread in threads]

In [301]:
print([(k,v) for (k,v) in sorted(dict(CO.degree()).items(), key=lambda x:x[1],reverse=True)][0])
print([(k,v) for (k,v) in sorted(dict(CR.degree()).items(), key=lambda x:x[1],reverse=True)][0])
print([(k,v) for (k,v) in sorted(dict(ME.degree()).items(), key=lambda x:x[1],reverse=True)][0])
print([(k,v) for (k,v) in sorted(dict(PL.degree()).items(), key=lambda x:x[1],reverse=True)][0])
print([(k,v) for (k,v) in sorted(dict(R.degree()).items(), key=lambda x:x[1],reverse=True)][0])


('57681616', 5)
('50175634', 5)
('22518391', 5)
('38332691', 5)
('38055575', 5)


In [878]:
len(list(set(posts_quotes)))

21228

In [ ]:
degree_seeds

In [153]:
crime_nodes

("Rittenhouse:Not one, but TWO 'hood' mobs attacking innocents in one of the best neighborhoods in the city. Real nice.We all know the demographics of said mobs, I just wish the media and law enforcement would say it.",
 '50175634')

In [154]:
metro_nodes

('rotodome made a statement that you had a better chance to die in an auto accident then to be murdered in Philly. The stats in 2010 are as follows..People killed in auto accidents in the city- 84people murdered in the city- 306 (2011 was 324)so to sum it up. You are approximately 4 times as likely to be murdered in Philly then to die in an auto accident.. Interesting..',
 '22518391')

In [155]:
plan_nodes

('Well on the W lot at 15th and Chestnut is now permanently closed - fencing is up and an under progress sign with the W render now hungwow',
 '38332691')

In [156]:
retail_nodes

('This is what I got so far:1. CHIPOTLE Chipotle is coming to Liberty Place. If you go to the food court, you will see a boarded-up area near the corner across-ish from Chic-Fil-A. 2. INDOCHINO Just across from Liberty Place, Indochino is officially opened. It is located to the left of UNIQLO. It used to be occupied by Liberty Travel.3. LIBERTY TRAVEL Speaking of Liberty Travel, they moved one block away, closer to Modell\'s. I personally love how they make the building kinda chic, clean and attractive from outside. There is a huge monitor display inside that made people turned their heads around to see what\'s up. And on the top of that, this means one more empty storefront is filled. YAY!4. NEW BALANCE I heard New Balance concept store is coming to Walnut this Summer (before Under Armour!) How stealthy! 5. BANANA REPUBLIC FACTORY STORE AND GAP OUTLET STORE Banana republic Factory store has been opened for 3 weeks now on Chestnut. And I am glad the GAP outlet across the street from it

In [883]:
len(list(set(citydata_df.post_id.tolist() + citydata_df.quote_id.tolist())))

23058

In [31]:
titles= """How's everyone doing amongst the Coronavirus shut down?
Official Greater Philadelphia Area Crime Thread (Chester, New Castle: 2013, middle school, university) Official Philadelphia Metro Crime Thread (York, Chester: apartment complexes, houses, unemployment) 
Philadelphia 2035 (Houston: foreclosure, neighborhoods, wage) 
Retail coming to Philadelphia (Penn, Burlington: real estate, house, buying)"""

title_embeddings = smodel.encode(titles.split('\n'))

title_terms = [text_tokenize(title) for title in titles.split('\n')]

In [79]:

df0 = cluster_thread_embeddings_kmeans_tfidf(threads,
                                             city_embeddings,
                                             vectorizer='tfidf',
                                             mindf=10,
                                             clusters=4,ngrams=(1,2),
                                             kbest=7500)

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



-5.476321141738323


In [26]:
#no priors thread level

df1 = cluster_thread_embeddings_kmeans_tfidf(threads,
                                                                                                                           city_embeddings,
                                                                       mindf=10,
                                                                                    clusters=4,ngrams=(1,3),
                                                                                 kbest=7500)

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



-4.457882055127163


In [ ]:
df1

In [35]:
#cluster center priors from high degree nodes


df2 = cluster_thread_embeddings_kmeans_tfidf(threads, 
                                             city_embeddings,
                                             clusters=4,
                                             ngrams=(1,3),
                                             mindf=10,
                                             kbest=7500,init_=degree_seed_embeddings)

setting cluster centers


C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:1362: RuntimeWarning:

Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.



-4.266071728654636


In [36]:
#cluster center priors from first post
#threads


df3 = cluster_thread_embeddings_kmeans_tfidf(threads, 
                                            city_embeddings,
                                            mindf=10,
                                            clusters=4,
                                            ngrams=(1,3),
                                            kbest=7500,init_=first_posts_embeddings)

setting cluster centers


C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:1362: RuntimeWarning:

Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.



-4.57219736559069


## Post-Level Embeddings

In [15]:
posts_quotes = citydata_df.post.tolist() + [quote for quote in citydata_df.quote.tolist() if quote not in citydata_df.post.tolist()]
#post_embeddings = smodel.encode(posts_quotes)


In [37]:
#post-level embeddings no priors
df4 = cluster_thread_embeddings_kmeans_tfidf(posts_quotes, 
                                                                                                                           post_embeddings,
                                                                       mindf=10,
                                                                                  clusters=4,ngrams=(1,3),
                                                                                 kbest=7500)

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



-3.7146329029264553


In [38]:
#posts_quotes = citydata_df.post.tolist() + [quote for quote in citydata_df.quote.tolist() if quote not in city_data_df.post.tolist()]
#post_embeddings = smodel.encode(posts_quotes)
#post-level topics with init_posts as centers
df5 = cluster_thread_embeddings_kmeans_tfidf(posts_quotes, 
                                                                                                                           post_embeddings,
                                                                       mindf=10,
                                                                                    clusters=4,ngrams=(1,3),
                                                                                 kbest=7500,init_=first_posts_embeddings)

setting cluster centers


C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:1362: RuntimeWarning:

Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.



-2.562278630401623


In [39]:
#post-level embeddings degree seeds
df6 = cluster_thread_embeddings_kmeans_tfidf(posts_quotes, 
                                                                                                                           post_embeddings,
                                                                       mindf=10,
                                                                                    clusters=4,ngrams=(1,3),
                                                                                 kbest=7500,init_=degree_seeds)

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



-3.288412726815769


In [819]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=10)
v = tsne.fit_transform(city_embeddings)


In [822]:
from plotly.offline import iplot

plot(px.scatter(x=v[:,0],y=v[:,1], color=[str(k) for k in km8.labels_]))

'temp-plot.html'

In [40]:
#post-level embeddings froum title seeds
df7 = cluster_thread_embeddings_kmeans_tfidf(posts_quotes, 
                                                                                    post_embeddings,
                                                                       mindf=10,
                                                                                    clusters=4,ngrams=(1,3),
                                                                                 kbest=7500,init_=title_embeddings)

setting cluster centers


C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:1362: RuntimeWarning:

Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.



-2.761954646512221


In [41]:
#thread-level embedding forum title seeds
df8 = cluster_thread_embeddings_kmeans_tfidf(threads, 
                                                                                    city_embeddings,
                                                                       mindf=10,
                                                                                clusters=4,ngrams=(1,3),
                                                                                 kbest=7500,init_=title_embeddings)

setting cluster centers


C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\rmomi\anaconda3\envs\ktransformers\Lib\site-packages\sklearn\cluster\_kmeans.py:1362: RuntimeWarning:

Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.



-4.842381752972228


In [47]:
df8.keys()

dict_keys(['threads', 'term_weights', 'vectorizer', 'clusterer', 'topics'])

In [ ]:
['topics']

## Sentence Embeddings Evaluation

In [48]:
se_topics_eval_df = pd.concat([
        get_coherence_diversity_scores(df4['threads'].thread, df4['topics'], fast,topn=25,topic_type='tfidf post'),
        get_coherence_diversity_scores(df7['threads'].thread, df7['topics'], fast,topn=25,topic_type='guided tfidf (post titles)'),
        get_coherence_diversity_scores(df5['threads'].thread, df5['topics'], fast,topn=25,topic_type='guided tfidf (initial posts)'),
        get_coherence_diversity_scores(df6['threads'].thread, df6['topics'], fast,topn=25,topic_type='guided tfidf (high degree)'),
        get_coherence_diversity_scores(df1['threads'].thread, df1['topics'], fast,topn=25,topic_type='tfidf threads'),
        get_coherence_diversity_scores(df8['threads'].thread, df8['topics'], fast,topn=25,topic_type='guided tfidf threads post titles'),
        get_coherence_diversity_scores(df3['threads'].thread, df3['topics'], fast,topn=25,topic_type='guided tfidf threads (initial posts)'),
        get_coherence_diversity_scores(df2['threads'].thread, df2['topics'], fast,topn=25,topic_type='guided tfidf threads (high degree)'),
        
          ])

In [49]:

se_topics_eval_df[['u_mass_coherence','puw:','jd:','we-cd:']]

,u_mass_coherence,puw:,jd:,we-cd:
tfidf post,-3.714633,0.73,0.851137,0.101701
guided tfidf (post titles),-2.761955,0.54,0.638784,0.065617
guided tfidf (initial posts),-2.562279,0.52,0.620674,0.071358
guided tfidf (high degree),-3.288413,0.74,0.857064,0.101701
tfidf threads,-4.457882,0.73,0.855737,0.234158
guided tfidf threads post titles,-4.842382,0.69,0.797508,0.065617
guided tfidf threads (initial posts),-4.572197,0.67,0.785500,0.218687
guided tfidf threads (high degree),-4.266072,0.67,0.785500,0.216487


In [50]:
se_topics_eval_df.mean()

u_mass_coherence   -3.808226
puw:                0.661250
jd:                 0.773988
we-pd:              0.667342
we-cd:              0.134416
we-irbo p=0.5:      0.600381
we-irbo p=0.9:      0.522834
dtype: float64

In [854]:
pd.DataFrame(se_topics_eval_df['we-cd:'])

,we-cd:
tfidf post,0.440207
guided tfidf (post titles),0.097374
guided tfidf (initial posts),0.085735
guided tfidf (high degree),0.085735
tfidf threads,0.215439
guided tfidf threads post titles,0.092573
guided tfidf threads (initial posts),0.185650
guided tfidf threads (high degree),0.232007


In [847]:
se_topics_eval_df['u_mass_coherence'][:4].mean() 

-3.0751703998894153

In [848]:
se_topics_eval_df['u_mass_coherence'][4:].mean()

-5.2605348650148365

## Gensim Post and Thread Models

In [51]:
pposts = df7['threads'].thread
pthreads = df1['threads'].thread
gensim_post_model, gensim_post_terms, gensim_post_corpus, gensim_processed_posts = gensim_lda(pposts, topic_num=4)
gensim_thread_model, gensim_thread_terms, gensim_thread_corpus, gensim_processed_threads = gensim_lda(pthreads, topic_num=4)

In [54]:
first_posts_tokens = [text_tokenize(post) for post in first_posts]

In [ ]:
gensim_processed_posts

## Gensim Guided Initial Post

In [56]:
gensim_init_post_model, gensim_init_posts_terms, gensim_init_posts_corpus, processed_posts_init_posts = gensim_lda(pposts, topic_num=4, topic_word_priors=first_posts_tokens, eta_=1.0)


In [57]:
gensim_thread_init_post_model, gensim_thread_init_posts_terms, gensim__thread_init_posts_corpus, gensim_thread_processed_posts_init_posts = gensim_lda(pthreads, topic_num=4, topic_word_priors=first_posts_tokens, eta_=1.0)


## Gensim Guided Titles

In [58]:
gensim_titles_model, gensim_titles_terms, gensim_titles_corpus, gensim_titles_processed_posts = gensim_lda(pposts, topic_num=4, topic_word_priors=title_terms, eta_=1.0)

In [59]:
gensim_thread_titles_model, gensim_thread_titles_terms, gensim_thread_titles_corpus, gensim_thread_titles_processed_posts = gensim_lda(pthreads, topic_num=4, topic_word_priors=title_terms, eta_=1.0)

## Gensim Guided High Degree

In [60]:
degree_priors = [text_tokenize(node) for node in [covid_nodes, crime_nodes + metro_nodes, plan_nodes,retail_nodes]]

In [61]:
gensim_degree_model, gensim_degree_terms, gensim_degree_corpus, gensim_degree_processed_posts = gensim_lda(pposts, topic_num=4, topic_word_priors=degree_priors, eta_=1.0)

In [62]:
gensim_thread_degree_model, gensim__thread_degree_terms, gensim_thread_degree_corpus, gensim_thread_degree_processed_posts = gensim_lda(pthreads, topic_num=4, topic_word_priors=degree_priors, eta_=1.0)

In [ ]:
pd.DataFrame([g[:10] for g in topics6])

In [863]:
pd.DataFrame([g[:10] for g in gensim_degree_terms])

,0,1,2,3,4,5,6,7,8,9
0,money,state,local,people,issue,care,neighborhood,income,help,white
1,store,retail,mall,shopping,retailer,location,center,shop,gallery,also
2,city,think,philadelphia,philly,people,year,time,even,area,much
3,street,market,building,walnut,space,chestnut,center,east,block,south


In [ ]:
gensim_degree_corpus[1]

In [870]:
len(df7)

23436

In [873]:
len(list(set(df7.thread)))

20582

## Gensim Evaluations

In [874]:
len(city_embeddings)

15008

In [63]:
gensim_eval_df = pd.concat([
    get_coherence_diversity_scores(gensim_processed_posts, gensim_post_terms,fast,metric='u_mass',topic_type='lda posts'),
    get_coherence_diversity_scores(gensim_titles_processed_posts, gensim_titles_terms,fast,metric='u_mass',topic_type='guided lda (post titles)'),
    get_coherence_diversity_scores(gensim_degree_processed_posts, gensim_degree_terms, fast, metric='u_mass',topic_type='guided lda (high degree)'),
    get_coherence_diversity_scores(processed_posts_init_posts, gensim_init_posts_terms,fast, metric="u_mass",topic_type='guided lda (initial posts)'),
    get_coherence_diversity_scores(gensim_processed_threads, gensim_thread_terms,fast,metric='u_mass',topic_type='lda threads'),
    get_coherence_diversity_scores(gensim_thread_degree_processed_posts, gensim__thread_degree_terms,fast,metric='u_mass',topic_type='lda threads (high degree)'),
    get_coherence_diversity_scores(gensim_thread_titles_processed_posts, gensim_thread_titles_terms,fast,metric='u_mass',topic_type='lda threads (post titles)'),
    get_coherence_diversity_scores(gensim_thread_processed_posts_init_posts, gensim_thread_init_posts_terms,fast,metric='u_mass',topic_type='lda threads (initial post)'),
])

In [65]:
gensim_eval_df[['u_mass_coherence','puw:','jd:','we-cd:']]

,u_mass_coherence,puw:,jd:,we-cd:
lda posts,-2.132370,0.975,0.948954,0.395534
guided lda (post titles),-2.125496,0.975,0.948511,0.394123
guided lda (high degree),-2.358853,0.975,0.962835,0.345177
guided lda (initial posts),-2.093461,0.950,0.944503,0.428048
lda threads,-2.131009,0.900,0.934787,0.341057
lda threads (high degree),-2.063384,0.900,0.931102,0.341057
lda threads (post titles),-2.132315,0.900,0.949102,0.341057
lda threads (initial post),-2.036049,0.925,0.942018,0.322595


In [937]:
gensim_eval_df

,u_mass_coherence,puw:,jd:,we-pd:,we-cd:,we-irbo p=0.5:,we-irbo p=0.9:
lda posts,-2.124201,0.975,0.948954,0.671730,0.395534,0.620888,0.612986
guided lda (post titles),-2.161198,0.975,0.948511,0.670787,0.394123,0.615967,0.612148
guided lda (high degree),-2.639306,1.000,0.980392,0.675528,0.310433,0.660869,0.610309
guided lda (initial posts),-2.072529,0.950,0.948188,0.667665,0.428048,0.530960,0.575025
lda threads,-2.131009,0.900,0.934787,0.671317,0.341057,0.573213,0.571095
lda threads (high degree),-2.063384,0.900,0.931102,0.674843,0.341057,0.607203,0.580633
lda threads (post titles),-2.132315,0.900,0.949102,0.668235,0.341057,0.577161,0.576079
lda threads (initial post),-2.036049,0.925,0.942018,0.670593,0.322595,0.576035,0.575838


In [922]:
gensim_eval_df['u_mass_coherence'][:4].mean()


-2.6938108251342445

In [923]:
gensim_eval_df['u_mass_coherence'][4:].mean()


-6.131735315521489

In [ ]:
pd.DataFrame(gensim_degree_terms[:25]).transpose()

In [ ]:
pd.DataFrame(gensim_post_terms[:25]).transpose()

In [ ]:
pd.DataFrame(gensim_init_posts_terms).transpose()

In [138]:
citydata_df.to_parquet('citydata.parquet')

In [523]:
crime_nodes + metro_nodes

"Rittenhouse:Not one, but TWO 'hood' mobs attacking innocents in one of the best neighborhoods in the city. Real nice.We all know the demographics of said mobs, I just wish the media and law enforcement would say it.rotodome made a statement that you had a better chance to die in an auto accident then to be murdered in Philly. The stats in 2010 are as follows..People killed in auto accidents in the city- 84people murdered in the city- 306 (2011 was 324)so to sum it up. You are approximately 4 times as likely to be murdered in Philly then to die in an auto accident.. Interesting.."

In [589]:
degree_priors = [text_tokenize(node) for node in [covid_nodes, crime_nodes + metro_nodes, plan_nodes,retail_nodes]]

In [557]:
first_posts = [init_posts[0], init_posts[1] + init_posts[2], init_posts[3], init_posts[4]]

In [585]:
first_posts_embeddings = smodel.encode(first_posts)
first_posts_tokens = [text_tokenize(post) for post in first_posts]

In [584]:
text_tokenize(first_posts[0])

['everything',
 'shutting',
 'slowly',
 'surely',
 'job',
 'shut',
 'rest',
 'month',
 'home',
 'lol',
 'still',
 'gotten',
 'grocery',
 'store',
 'knee',
 'replacement',
 'considered',
 'essential',
 'operation',
 'still',
 'going',
 'stay',
 'well',
 'guy']

In [458]:
ip = [text_process(post) for post in init_posts]
initial_posts = flatten_list([nltk.wordpunct_tokenize(i) for i in ip])


In [547]:
len(citydata_df), len(mvecs)

(15008, 15008)

In [548]:
plot(px.scatter(x=mvecs[:,0],y=mvecs[:,1],color=citydata_df.forum))

'temp-plot.html'

In [427]:
Counter(list(etas))

Counter({0.2: 36473, 0.4: 200})

In [428]:
model = gensim.models.ldamodel.LdaModel(
            corpus=bow, id2word=dictionary, num_topics=5,
            random_state=42, chunksize=100, eta=np.array(etas),
            eval_every=-1, update_every=1,
            passes=150, alpha='auto', per_word_topics=True)

In [429]:
model.show_topics()

[(0,
  '0.037*"retail" + 0.018*"building" + 0.014*"street" + 0.013*"old" + 0.013*"mall" + 0.012*"love" + 0.011*"space" + 0.011*"across" + 0.011*"location" + 0.010*"hope"'),
 (1,
  '0.020*"top" + 0.020*"luxury" + 0.019*"built" + 0.014*"company" + 0.013*"flagship" + 0.011*"story" + 0.011*"floor" + 0.010*"housing" + 0.010*"offer" + 0.009*"largest"'),
 (2,
  '0.018*"traffic" + 0.016*"news" + 0.013*"demand" + 0.012*"help" + 0.011*"tax" + 0.010*"amount" + 0.010*"income" + 0.009*"larger" + 0.009*"region" + 0.008*"poverty"'),
 (3,
  '0.030*"have" + 0.022*"will" + 0.019*"like" + 0.017*"more" + 0.017*"there" + 0.015*"one" + 0.014*"think" + 0.012*"city" + 0.011*"people" + 0.010*"can"'),
 (4,
  '0.124*"store" + 0.024*"food" + 0.022*"department" + 0.022*"opened" + 0.015*"chain" + 0.015*"outlet" + 0.014*"line" + 0.013*"fashion" + 0.009*"service" + 0.009*"district"')]

In [430]:
gterms = [[token for token in m[1].split('"') if not re.findall(r'\d',token)] for m in  model.show_topics(num_words=25)]

In [130]:
gs = [float(s) for s in """-2.590164 

-2.631312 

-2.104102 

-2.365565 

-2.107477 

-2.150025 

-2.324303 

-2.226696 """.split()]

In [131]:
cs =[float(s) for s in """-4.102533 

-2.761796 

-2.561912 

-2.530231 

-5.469968 

-4.842382 

-4.572197 

-4.266072 """.split()]

In [132]:
cs

[-4.102533,
 -2.761796,
 -2.561912,
 -2.530231,
 -5.469968,
 -4.842382,
 -4.572197,
 -4.266072]

In [133]:
np.mean(cs),np.std(cs)

(-3.8883863749999996, 1.0571740887541816)

In [134]:
plot(px.bar(x=range(len(cs)),y=cs))

'temp-plot.html'

In [110]:
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

import numpy as np
import pandas as pd
import scipy


trace1 = go.Box(
    y=cs,
    name = 'Box Plot',
    boxpoints='all',
    jitter=0.3,
    marker = dict(
        color = 'rgb(214,12,140)',
    ),
)
trace2 = go.Box(
    y=gs,
    name = 'Box Plot',
    boxpoints='all',
    jitter=0.3,
    marker = dict(
        color = 'rgb(14,128,140)',
    ),
)
layout = go.Layout(
    width=500,
    yaxis=dict(
        title='Topical Coherence Scores',
        zeroline=False
    ),
)

data = [trace1,trace2]
fig= go.Figure(layout=layout)
fig.add_trace(trace1)
fig.add_trace(trace2)
plot(fig, filename='coherence scores')

'coherence scores.html'

In [135]:
c_scores = pd.DataFrame({'SE-Topical Coherence':cs, 'LDA Topical Coherence':gs})

In [136]:
pd.melt(c_scores)

,variable,value
0,SE-Topical Coherence,-4.102533
1,SE-Topical Coherence,-2.761796
2,SE-Topical Coherence,-2.561912
3,SE-Topical Coherence,-2.530231
4,SE-Topical Coherence,-5.469968
5,SE-Topical Coherence,-4.842382
6,SE-Topical Coherence,-4.572197
7,SE-Topical Coherence,-4.266072
8,LDA Topical Coherence,-2.590164
9,LDA Topical Coherence,-2.631312


In [137]:
plot(px.box(pd.melt(c_scores), x='variable',y='value',points='all',color='variable',title='Topical Coherence Scores'))

'temp-plot.html'

In [127]:
dff = pd.DataFrame()

dff['se_puw'] = [float(s) for s in """0.74 0.54 0.52 0.54 0.74 0.69 0.67 0.67 """.split()]

dff['se_jd'] = [float(s) for s in """0.857064 0.638784 0.620674 0.639471 0.867287 0.797508 0.785500 0.785500 """.split()]

dff['se_wecd'] = [float(s) for s in """0.060969 0.065617 0.071358 0.069196 0.129490 0.065617 0.218687 0.216487 """.split()]

dff['lda_puw'] = [float(s) for s in """1.000 1.000 0.950 0.975 0.950 0.950 0.950 0.900 """.split()]

dff['lda_jd'] = [float(s) for s in """0.959436 0.959575 0.959861 0.966373 0.959436 0.959436 0.959436 0.948797 """.split()]

dff['lda_wecd'] = [float(s) for s in """0.300081 0.277969 0.294762 0.262589 0.302670 0.313002 0.302670 0.323610 """.split()]


In [129]:



fig = plot(px.box(pd.melt(dff), x='variable',y='value',points='all',color='variable',title='Topical Diversity Scores (PUW, JD, WECD)'))


In [138]:
5.46 - 2.53

2.93

In [139]:
citydata_df

,post_id,post_body,post,datetime,quote_id,quote_body,quote,forum
0,758124,"td class=""alt1 altop"" id=""td_post_57581240"">\n...",So just about everything is shutting down slo...,2020-03-16 12:36:00,,,,coronavirus
1,758139,"td class=""alt1"" id=""td_post_57581398"">\n<!-- m...",I am so lucky to be able to work remotely thro...,2020-03-16 12:55:00,,,,coronavirus
2,758214,"td class=""alt1"" id=""td_post_57582145"">\n<!-- m...",I agree with really feeling for those in the s...,2020-03-16 14:26:00,,,,coronavirus
3,758228,"td class=""alt1"" id=""td_post_57582283"">\n<!-- m...",I'm working remotely as well. Went for booze y...,2020-03-16 14:45:00,,,,coronavirus
4,758250,"td class=""alt1"" id=""td_post_57582507"">\n<!-- m...",Working remotely for the foreseeable future. W...,2020-03-16 15:11:00,,,,coronavirus
...,...,...,...,...,...,...,...,...
4110,958699,"td class=""alt1"" id=""td_post_39586996"">\n<!-- m...",So are the rumors true that AT&T is looking to...,2015-05-12 13:56:00,,,,retail
4111,958730,"td class=""alt1"" id=""td_post_39587305"">\n<!-- m...",So are the rumors true that AT&T is looking t...,2015-05-12 14:15:00,39586996,"<td class=""alt2"" style=""border:1px inset"">\n<d...",So are the rumors true that AT&T is looking to...,retail
4112,958755,"td class=""alt1"" id=""td_post_39587559"">\n<!-- m...",That's the rumor still. Although now I read t...,2015-05-12 14:31:00,39587305,"<td class=""alt2"" style=""border:1px inset"">\n<d...",That's the rumor still. Although now I read th...,retail
4113,959147,"td class=""alt1"" id=""td_post_39591477"">\n<!-- m...","Too much vacancy and low end ""high end"" retail...",2015-05-12 19:47:00,,,,retail


In [140]:
pwd

'C:\\Users\\rmomi\\OneDrive\\Desktop\\city-data2'

In [141]:
ls

 Volume in drive C is Windows
 Volume Serial Number is A2DB-0107

 Directory of C:\Users\rmomi\OneDrive\Desktop\city-data2

11/08/2023  06:14 PM    <DIR>          .
11/01/2023  09:42 PM    <DIR>          ..
10/28/2023  10:43 PM    <DIR>          .ipynb_checkpoints
10/20/2023  11:57 AM    <DIR>          __pycache__
04/13/2022  05:41 PM        64,928,417 20newsgroups_tagged_df.pkl
10/06/2023  08:23 PM    <DIR>          amzhq
10/06/2023  08:23 PM    <DIR>          beingwhite
02/09/2020  07:46 AM         4,252,000 city_data_crime.csv
02/11/2020  07:34 AM               442 city_data_dscope.py
01/18/2020  10:26 AM        16,212,603 city_data_philly2035.csv
01/18/2020  01:51 PM        88,648,466 city_data_philly2035_dscope.pkl
01/19/2020  11:16 AM           678,499 city_data_philly2035_dscope_lats.pkl
01/19/2020  11:56 AM        72,623,418 city_data_philly2035_tagged_scored.csv
04/15/2020  08:58 PM            10,208 city_data_scrape.py
01/17/2020  11:40 AM         8,432,028 city_data0.csv
11/

In [143]:
seed_posts = ["""For every one of those articles (written by beer lovers), there is a scientific article that explains the facts of alcohol (carcinogen) and its negative health impacts. I have drank plenty in my life, and still enjoy a beer or wine, but I'm not fooling myself into thinking its a life-sustaining entity. The same people who complain about not making enough money or not being able to pay the rent will go out and spend their limited cash on booze.  How much of the fed's cash payments will go to alcohol?"""]

[citydata_df.iloc[i] for i in range(len(citydata_df)) if re.findall('For every one of those articles (written by beer lovers), there is a scientific article',citydata_df.iloc[i].post)]


[]

In [144]:
citydata_df

,post_id,post_body,post,datetime,quote_id,quote_body,quote,forum
0,758124,"td class=""alt1 altop"" id=""td_post_57581240"">\n...",So just about everything is shutting down slo...,2020-03-16 12:36:00,,,,coronavirus
1,758139,"td class=""alt1"" id=""td_post_57581398"">\n<!-- m...",I am so lucky to be able to work remotely thro...,2020-03-16 12:55:00,,,,coronavirus
2,758214,"td class=""alt1"" id=""td_post_57582145"">\n<!-- m...",I agree with really feeling for those in the s...,2020-03-16 14:26:00,,,,coronavirus
3,758228,"td class=""alt1"" id=""td_post_57582283"">\n<!-- m...",I'm working remotely as well. Went for booze y...,2020-03-16 14:45:00,,,,coronavirus
4,758250,"td class=""alt1"" id=""td_post_57582507"">\n<!-- m...",Working remotely for the foreseeable future. W...,2020-03-16 15:11:00,,,,coronavirus
...,...,...,...,...,...,...,...,...
4110,958699,"td class=""alt1"" id=""td_post_39586996"">\n<!-- m...",So are the rumors true that AT&T is looking to...,2015-05-12 13:56:00,,,,retail
4111,958730,"td class=""alt1"" id=""td_post_39587305"">\n<!-- m...",So are the rumors true that AT&T is looking t...,2015-05-12 14:15:00,39586996,"<td class=""alt2"" style=""border:1px inset"">\n<d...",So are the rumors true that AT&T is looking to...,retail
4112,958755,"td class=""alt1"" id=""td_post_39587559"">\n<!-- m...",That's the rumor still. Although now I read t...,2015-05-12 14:31:00,39587305,"<td class=""alt2"" style=""border:1px inset"">\n<d...",That's the rumor still. Although now I read th...,retail
4113,959147,"td class=""alt1"" id=""td_post_39591477"">\n<!-- m...","Too much vacancy and low end ""high end"" retail...",2015-05-12 19:47:00,,,,retail


In [147]:
covid.post_id.tolist()[0]

'758124'

In [146]:
crime.post_id.tolist()[0]

'908813'

In [148]:
metro.post_id.tolist()[0]

'251839'

In [149]:
plan.post_id.tolist()[0]

'958364'

In [150]:
retail.post_id.tolist()[0]

'710692'